Conferência entre Leads e Agendamentos




1.   Baixar os relatório de **Leads**, **Agendamentos** e **Vendas** do período agendado em Excel
2.   Abrir o ícone de pasta no menu lateral da esquerda <<<<<<<<
3.   Fazer upload dos arquivos
4.   Apertar o play e aguardar




### Carregar Bibliotecas

In [1]:
# Importações gerais
import os
import re
import unicodedata

# Manipulação de dados
import pandas as pd

### Bases

In [2]:
# Caminho da pasta onde estão os arquivos
folder_path = '/Users/luisfaria/Desktop/sEngineer/dash/db'
files = os.listdir(folder_path)

# Inicializar os DataFrames como None
df_agd = None
df_leads = None
df_vendas = None

# Caminho da pasta onde estão os arquivos
files = os.listdir(folder_path)

# Verificar os arquivos e carregar os que correspondem
for file_name in files:
    file_path = os.path.join(folder_path, file_name)

    # Verificar se é um arquivo Excel
    if file_name.endswith('.xlsx'):
        if 'appointments' in file_name.lower():
            df_agd = pd.read_excel(file_path)
            print(f"Arquivo de agendamentos carregado: {file_name}")
        elif 'leads' in file_name.lower():
            df_leads = pd.read_excel(file_path)
            print(f"Arquivo de leads carregado: {file_name}")
        elif 'sales' in file_name.lower():
            df_vendas = pd.read_excel(file_path)
            print(f"Arquivo de vendas carregado: {file_name}")
        # elif 'vendas' in file_name.lower():
        #     df_vendas2 = pd.read_excel(file_path)
        # elif'agenda' in file_name.lower():
        #     df_agd2 = pd.read_excel(file_path)
# Verificar se todos os DataFrames foram carregados
if df_agd is None:
    print("Nenhum arquivo de agendamentos encontrado.")
if df_leads is None:
    print("Nenhum arquivo de leads encontrado.")
if df_vendas is None:
    print("Nenhum arquivo de vendas encontrado.")

Arquivo de leads carregado: leads.xlsx
Arquivo de vendas carregado: sales.xlsx
Arquivo de agendamentos carregado: appointments.xlsx


In [3]:
# df_agd = pd.concat([df_agd, df_agd2], ignore_index=True)
# df_vendas = pd.concat([df_vendas, df_vendas2], ignore_index=True)

##### Outras bases

In [4]:
# # Outras bases
# folder_path = '/Users/luisfaria/Desktop/sEngineer/dash/db/more'
# files = os.listdir(folder_path)

# # Inicializar os DataFrames como None
# df_agd_02 = None
# df_vendas_02 = None

# # Verificar os arquivos e carregar os que correspondem
# for file_name in files:
#     file_path = os.path.join(folder_path, file_name)

#     # Verificar se é um arquivo Excel
#     if file_name.endswith('.xlsx'):
#         if 'appointments' in file_name.lower():
#             df_agd_02 = pd.read_excel(file_path)
#             print(f"Arquivo de agendamentos carregado: {file_name}")

#         elif 'sales' in file_name.lower():
#             df_vendas_02 = pd.read_excel(file_path)
#             print(f"Arquivo de vendas carregado: {file_name}")
# # Verificar se todos os DataFrames foram carregados
# if df_agd_02 is None:
#     print("Nenhum arquivo de agendamentos encontrado.")
# if df_leads is None:
#     print("Nenhum arquivo de leads encontrado.")
# if df_vendas_02 is None:
#     print("Nenhum arquivo de vendas encontrado.")

In [5]:
# df_agd_concat = pd.concat([df_agd, df_agd_02], ignore_index=True)
# df_sales_concat = pd.concat([df_vendas, df_vendas_02], ignore_index=True)

In [6]:
# df_agd = df_agd_concat.copy()
# df_vendas = df_sales_concat.copy()
# df_agd.shape

### Leads x Atendidos

#### Helpers

In [7]:
# Função para limpar telefones
def clean_telephone(telefone):
    # Remove todos os caracteres não numéricos e garante que o valor é uma string
    telefone = re.sub(r'[^\d]', '', str(telefone))
    # Remove o prefixo +55, caso exista
    if telefone.startswith('55'):
        telefone = telefone[2:]
    return telefone

#### Appointments Treatments

In [8]:
# Classificação dos agendamentos
# Criação de colunas auxiliares para categorizar os status
df_agd["proced_avaliação"] = df_agd['Procedimento'].str.contains(r'.*AVALIAÇÃO.*', case=False)
df_agd["agendamento"] = df_agd['Status'].str.contains(r'.*Atendido|Falta.*', case=False)
df_agd["comparecimento"] = df_agd['Status'].str.contains(r'.*Atendido.*', case=False)
df_agd["esta agendado?"] = df_agd['Status'].str.contains("Agendado", case=False)
df_agd["falta ou cancelado?"] = df_agd['Status'].str.contains(r'.*Falta|Cancelado.*', case=False)

# Filtrando agendamentos com procedimento de avaliação e status "Atendido + Falta"
df_agendamentos_aval = df_agd[
    (df_agd['proced_avaliação'] == True) &
    (df_agd['agendamento'] == True)
]

# Filtrando comparecimentos com procedimento de avaliação e status apenas "Atendido"
df_comparecimentos_aval = df_agd[
    (df_agd['proced_avaliação'] == True) &
    (df_agd['agendamento'] == True) &
    (df_agd['comparecimento'] == True)
]

# Lista de procedimentos de interesse
procedimentos_que_vamos_olhar = [
    'AVALIAÇÃO TATUAGEM',
    'AVALIAÇÃO INJETÁVEIS E INVASIVOS',
    'AVALIAÇÃO ESTÉTICA',
    'AVALIAÇÃO DEPILAÇÃO',
    'AVALIAÇÃO MEDICINA ESTÉTICA'
]

# Filtrando apenas os procedimentos de interesse nos dataframes de agendamentos e comparecimentos
df_agendamentos_aval = df_agendamentos_aval.loc[
    df_agendamentos_aval['Procedimento'].isin(procedimentos_que_vamos_olhar)
]
df_comparecimentos_aval = df_comparecimentos_aval.loc[
    df_comparecimentos_aval['Procedimento'].isin(procedimentos_que_vamos_olhar)
]

# Tratamento de dados no df_comparecimentos_aval
df_comparecimentos_aval['Telefone'] = df_comparecimentos_aval['Telefone'].fillna('Cliente sem telefone')
df_comparecimentos_aval['Email'] = df_comparecimentos_aval['Email'].fillna('NA')
df_comparecimentos_aval['Telefone'] = df_comparecimentos_aval['Telefone'].astype(str)

# Criar lista de telefones limpos para múltiplos números separados por "/"
df_comparecimentos_aval['Telefones Limpos'] = df_comparecimentos_aval['Telefone'].apply(
    lambda x: [clean_telephone(num) for num in str(x).split('/')]
)

# Criar sets de telefones e emails agendados para busca rápida
telefones_agendados = set(num for sublist in df_comparecimentos_aval['Telefones Limpos'] for num in sublist)
emails_agendados = set(df_comparecimentos_aval['Email'])

# Limpar telefones no df_agd, incluindo separar múltiplos telefones
df_agd['Telefone'] = df_agd['Telefone'].fillna('Cliente sem telefone')
df_agd['Telefone'] = df_agd['Telefone'].astype(str)
df_agd['Telefones Limpos'] = df_agd['Telefone'].apply(lambda x: [clean_telephone(num) for num in str(x).split('/')])

# Verificação de status
df_agd["falta ou cancelado?"] = df_agd['Status'].str.contains(r'.*Falta|Cancelado.*', case=False, na=False)
df_agd["esta agendado?"] = df_agd['Status'].str.contains(r'(?<!Re)Agendado', case=False, na=False)

procedimentos_que_vamos_olhar = ['AVALIAÇÃO TATUAGEM', 'AVALIAÇÃO INJETÁVEIS E INVASIVOS','AVALIAÇÃO ESTÉTICA', 'AVALIAÇÃO DEPILAÇÃO', 'AVALIAÇÃO MEDICINA ESTÉTICA']

# Filtrar os leads que estão agendados ou falta/cancelado
df_agd_filtrado = df_agd[(df_agd["esta agendado?"] == True) | (df_agd["falta ou cancelado?"] == True)]
df_agd_filtrado = df_agd_filtrado.loc[df_agd_filtrado['Procedimento'].isin(procedimentos_que_vamos_olhar)]

#### Sales Treatments

In [9]:
#Filtros
df_vendas = df_vendas.loc[df_vendas['Status'] == 'Finalizado']
df_vendas = df_vendas.loc[df_vendas['Consultor'] != 'BKO VENDAS']
df_vendas = df_vendas.loc[df_vendas['Unidade'] != 'PRAIA GRANDE']

# Convert all values to strings, replace commas with dots, and then convert to float
# df_vendas['Valor líquido'] = df_vendas['Valor líquido'].astype(str).str.replace(',', '.').astype(float)

# Convert all values to strings, replace commas with dots, handle non-numeric values, and then convert to float
df_vendas['Valor líquido'] = pd.to_numeric(df_vendas['Valor líquido'].astype(str).str.replace(',', '.'), errors='coerce')

# df_vendas.groupby('Unidade').agg({'Valor líquido': 'sum'})
df_vendas['Valor líquido'].sum()

# df_vendas treatment
df_vendas['Telefone(s) do cliente'] = df_vendas['Telefone(s) do cliente'].fillna('Cliente sem telefone')
df_vendas['Email do cliente'] = df_vendas['Email do cliente'].fillna('Cliente sem e-mail')
df_vendas['Telefone(s) do cliente'] = df_vendas['Telefone(s) do cliente'].astype(str)
df_vendas['Telefones Limpos'] = df_vendas['Telefone(s) do cliente'].apply(lambda x: [clean_telephone(num) for num in str(x).split('/')])

colunas_reduzido =  [
                    'Telefones Limpos',
                    'Telefone(s) do cliente',
                    'ID orçamento',
                    'Data venda',
                    'Unidade',
                    'Valor líquido'
]

#### Leads Treatments

In [10]:
import pandas as pd 
# Filtrando apenas unidades de estética
# df_leads = df_leads.loc[df_leads['Unidade'] != 'PLÁSTICA']
# df_leads = df_leads.loc[df_leads['Unidade'] != 'HOMA']

# Tratamento de dados no df_leads
df_leads['Telefone do lead'] = df_leads['Telefone do lead'].fillna('Cliente sem telefone')
df_leads['Email do lead'] = df_leads['Email do lead'].fillna('None')
df_leads['Telefone do lead'] = df_leads['Telefone do lead'].astype(str).apply(clean_telephone)

### Leads x Appointment "Atendido"

In [11]:
# Função para verificar se o lead foi agendado e retornar dados adicionais do df_comparecimentos_aval
def eh_comparecimento(row):
    telefone = row['Telefone do lead']
    email = row['Email do lead']

    # Encontrar match no dataframe de comparecimentos
    match = df_comparecimentos_aval[
        (df_comparecimentos_aval['Telefones Limpos'].apply(lambda x: telefone in x)) |
        (df_comparecimentos_aval['Email'] == email)
    ]

    if not match.empty:
        data = match.iloc[0]['Data']
        proced_avaliação = match.iloc[0]['proced_avaliação']
        agendamento = match.iloc[0]['agendamento']
        comparecimento = match.iloc[0]['comparecimento']
        unidade = match.iloc[0]['Unidade do agendamento']

        return pd.Series({
            'data_agenda': data,
            'proced_avaliação': proced_avaliação,
            'agendamento': agendamento,
            'comparecimento': comparecimento,
            'procedimento': comparecimento,
            'unidade_atd': unidade
        })

    return pd.Series({'data_agenda': None,
                    'proced_avaliação': None,
                    'agendamento': None,
                    'comparecimento': None,
                    'unidade_atd': None})

df_leads = df_leads.join(df_leads.apply(eh_comparecimento, axis=1))

#### Leads x Appointments - Final Treat

In [12]:
colunas_que_quero = ['ID do lead', 'Email do lead', 'Telefone do lead', 'Mensagem', 'Unidade', 
                    'Fonte','Dia da entrada', 'Source', 'Medium', 'Term', 'Content', 'Campaign', 
                    # Agendamentos
                     'unidade_atd', 'proced_avaliação', 'agendamento','comparecimento', 'data_agenda', 'procedimento']

df_leads = df_leads[colunas_que_quero]

# substituindo os null values por "não compareceu"
df_leads['comparecimento'] = df_leads['comparecimento'].fillna('Não compareceu')

In [13]:
# Agrupando por Unidade + Fonte
# # Reset the index to convert the Series to a DataFrame
groupby_unidade_fonte = df_leads.groupby(['Unidade', 'Fonte','comparecimento'])['ID do lead'].nunique().reset_index(name='count')

pivot_unidade_fonte = groupby_unidade_fonte.pivot_table(index=['Unidade','Fonte'],
                                                        columns=['comparecimento'],
                                                        values=['count'], # Use 'count' as the values column
                                                        fill_value=0,

                                                        )

pivot_unidade_fonte.columns = ['Compareceu', 'Não Compareceu']

### Leads x Outros Status

In [14]:
# separando leads que não compareceram para ver se estão agendados ou falta/cancelados
df_leads_que_nao_compareceram = df_leads.loc[df_leads['comparecimento'] == 'Não compareceu']

# Limpar telefones no df_leads_que_nao_compareceram
df_leads_que_nao_compareceram['Telefone do lead'] = df_leads_que_nao_compareceram['Telefone do lead'].apply(clean_telephone)

# Função para verificar se o lead está no df_agd_filtrado e trazer as informações adicionais
def verificar_status(row):
    telefone = row['Telefone do lead']
    email = row['Email do lead']
    match = df_agd_filtrado[(df_agd_filtrado['Telefones Limpos'].apply(lambda x: telefone in x)) | (df_agd_filtrado['Email'] == email)]

    if not match.empty:
        data = match.iloc[0]['Data']
        proced_avaliação = match.iloc[0]['proced_avaliação']
        agendamento = match.iloc[0]['esta agendado?']
        comparecimento = match.iloc[0]['falta ou cancelado?']
        status = match.iloc[0]['Status']
        procedimento = match.iloc[0]['Procedimento']
        unidade = match.iloc[0]['Unidade do agendamento']
        
        return pd.Series({
            'data_agenda_novo': data,
            'proced_avaliação_novo': proced_avaliação,
            'agendamento_novo': agendamento,
            'comparecimento_novo': comparecimento,
            'status_novo': status,
            'procedimento_novo': procedimento,
            'unidade_other': unidade
        })

    return pd.Series({
        'data_agenda_novo': None,
        'proced_avaliação_novo': None,
        'agendamento_novo': None,
        'comparecimento_novo': None,
        'status_novo': None,
        'procedimento_novo': None,
        'unidade_other': None,
    })

# Aplicar a função no df_leads_que_nao_compareceram e expandir a saída
df_leads_que_nao_compareceram = df_leads_que_nao_compareceram.join(df_leads_que_nao_compareceram.apply(verificar_status, axis=1))



/var/folders/c4/t_n58l316yl308hxjf5jlwsh0000gn/T/ipykernel_71094/874332515.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_leads_que_nao_compareceram['Telefone do lead'] = df_leads_que_nao_compareceram['Telefone do lead'].apply(clean_telephone)


### Creating df_leads_copy

In [15]:
# Copiar o df_leads original
df_leads_copy = df_leads.copy()

# Selecionar os leads que não compareceram
df_leads_que_nao_compareceram2 = df_leads_copy.loc[df_leads_copy['comparecimento'] == 'Não compareceu']

# Atualizar a coluna 'comparecimento' com os valores de 'status_novo' em df_leads_que_nao_compareceram
df_leads_que_nao_compareceram.loc[:, 'comparecimento'] = df_leads_que_nao_compareceram['status_novo']

# Atualizar o DataFrame original
df_leads_copy.update(df_leads_que_nao_compareceram)

In [16]:
df_leads_copy.loc[df_leads_copy['comparecimento'] == 'Não compareceu', 'comparecimento'] = 'Não está na agenda'
df_leads_copy.loc[df_leads_copy['comparecimento'] == True, 'comparecimento'] = 'Atendido'

In [17]:
df_leads_copy.drop(columns=['proced_avaliação', 'agendamento'], inplace=True)

df_leads_copy.columns = [
                     'ID do lead', 'Email do lead', 'Telefone do lead', 'Mensagem', 'Unidade', 
                     'Fonte', 'Dia da entrada', 'Source','Medium', 'Term', 'Content', 'Campaign', 
                     'unidade_agenda', 'status_agenda', 'data_na_agenda', 'procedimento']

In [18]:
df_leads_copy.columns

Index(['ID do lead', 'Email do lead', 'Telefone do lead', 'Mensagem',
       'Unidade', 'Fonte', 'Dia da entrada', 'Source', 'Medium', 'Term',
       'Content', 'Campaign', 'unidade_agenda', 'status_agenda',
       'data_na_agenda', 'procedimento'],
      dtype='object')

In [19]:
df_leads_copy['Unidade'].value_counts()

Unidade
PLÁSTICA        3268
HOMA            2676
JARDINS         1230
IPIRANGA         564
SANTOS           563
TATUAPÉ          520
COPACABANA       517
LONDRINA         485
OSASCO           464
CAMPINAS         419
ALPHAVILLE       403
LAPA             402
CENTRAL          395
MOEMA            394
SANTO AMARO      381
SOROCABA         380
TUCURUVI         355
MOOCA            347
TIJUCA           341
PRAIA GRANDE     317
SÃO BERNARDO     299
ITAIM            271
GUARULHOS        212
VILA MASCOTE     167
Name: count, dtype: int64

### Sales Treatments

In [20]:
df_vendas_teste = df_vendas[colunas_reduzido]

# Explode 'Telefones Limpos'
df_vendas_teste = df_vendas_teste.explode('Telefones Limpos')

# Fill valor líquido na 0
df_vendas_teste['Valor líquido'] = df_vendas_teste['Valor líquido'].fillna(0)

# Sort valor líquido highest to lowest
df_vendas_teste = df_vendas_teste.sort_values(by='Valor líquido', ascending=False)

In [21]:
df_vendas['Status'].value_counts()

Status
Finalizado    2406
Name: count, dtype: int64

In [22]:
df_vendas.loc[df_vendas['Nome cliente'] == 'Adriana Ramos Sousa']

,ID orçamento,Data orçamento,Status,Revenda,Data venda,Mês venda,Hora venda,Unidade,Consultor,Avaliador,...,Fonte do cadastro do cliente,Nome cliente,CPF cliente,ID cliente,Email do cliente,Telefone(s) do cliente,Data nascimento cliente,Profissão cliente,Cortesia?,Telefones Limpos


In [23]:
df_vendas_teste['qtd_orcamentos'] = df_vendas_teste['Telefones Limpos'].map(
    df_vendas_teste.groupby('Telefones Limpos')['ID orçamento'].nunique()
)

In [24]:
df_vendas_teste['valor_total_cliente'] = df_vendas_teste['Telefones Limpos'].map(
    df_vendas_teste.groupby('Telefones Limpos')['Valor líquido'].sum()
)
df_vendas_teste

,Telefones Limpos,Telefone(s) do cliente,ID orçamento,Data venda,Unidade,Valor líquido,qtd_orcamentos,valor_total_cliente
2836,11999812120,11999812120,438092,2025-03-19 16:48:37,MOEMA,24500.04,1,24500.04
1315,11998820400,11998820400,436540,2025-03-12 11:21:32,TATUAPÉ,23450.00,1,23450.00
2587,19998744865,19998744865,437775,2025-03-18 13:14:52,JARDINS,21200.04,1,21200.04
2672,11983741820,11983741820,437880,2025-03-18 17:50:32,MOEMA,21199.96,2,27719.96
614,11999705008,11999705008,435819,2025-03-07 13:53:27,SANTO AMARO,20000.00,1,20000.00
...,...,...,...,...,...,...,...,...
1928,1332236054,1378023981 / 1330194778 / 1332236054 / 1398101...,437144,2025-03-14 15:17:24,SANTOS,0.00,1,932.00
1928,1330194778,1378023981 / 1330194778 / 1332236054 / 1398101...,437144,2025-03-14 15:17:24,SANTOS,0.00,1,932.00
1928,1378023981,1378023981 / 1330194778 / 1332236054 / 1398101...,437144,2025-03-14 15:17:24,SANTOS,0.00,1,932.00
955,11949542652,11949542652,436136,2025-03-10 10:40:45,SÃO BERNARDO,0.00,1,90.00


In [25]:
# Remove duplicates by 'Telefones Limpos', keeping first
df_vendas_teste = df_vendas_teste.drop_duplicates(subset='Telefones Limpos', keep='first')

In [26]:
df_vendas_teste.loc[df_vendas_teste['Telefones Limpos'] == '11987437823']

,Telefones Limpos,Telefone(s) do cliente,ID orçamento,Data venda,Unidade,Valor líquido,qtd_orcamentos,valor_total_cliente


In [27]:
# Merge the exploded DataFrame with df_leads
df_leads_compras = pd.merge(
    df_leads_copy,
    df_vendas_teste,
    left_on='Telefone do lead',
    right_on='Telefones Limpos',
    how='left'
)

# Fill NaNs with 'Não é lead' for cases where no match is found
df_leads_compras['Unidade_y'] = df_leads_compras['Unidade_y'].fillna('Não é lead')

In [28]:
# Droping duplicated ID do lead
df_leads_compras = df_leads_compras.drop_duplicates(subset='ID do lead', keep='first')
df_leads_compras['Valor líquido'].sum()

264654.85

In [29]:
# df_leads_compras.groupby('Unidade_x').agg({'Valor líquido': 'sum'}).sort_values(by='Valor líquido', ascending=False).reset_index()

### Categorias

In [30]:
# Dicionário de mapeamento: chave é a palavra-chave, valor é a categoria
category_mapping = {
    'Preenchimento': 'Preenchimento',
    'Botox': 'Botox',
    'Ultraformer': 'Ultraformer',
    'Enzimas': 'Enzimas',
    'Lavieen': 'Lavieen',
    'Sculptra': 'Bioestimulador',
    'Bioestimulador': 'Bioestimulador',
    'Institucional': 'Institucional',
    'Crio': 'Crio',
    'Limpeza': 'Limpeza',
    'olheiras': 'Preenchimento',
    'prolipo' : 'Enzimas',
    'Prolipo' : 'Enzimas',
    'rugas' : 'Botox'
}

In [31]:
# Transforming ['Content'] to str
df_leads_compras['Content'] = df_leads_compras['Content'].astype(str)
df_leads_compras['Mensagem'] = df_leads_compras['Mensagem'].astype(str)

# Função para categorizar com base nas palavras-chave
def categorize(text):
    for keyword, category in category_mapping.items():
        if pd.notna(text) and keyword.lower() in text.lower():  # Verifica se a palavra-chave está presente
            return category
    return 'Indefinido'  # Categoria padrão se não encontrar correspondência

# Passo 1: Aplicar a função de categorização na coluna 'Content'
df_leads_compras['Categoria'] = df_leads_compras['Content'].apply(categorize)

# Passo 2: Aplicar a função de categorização na coluna 'Mensagem', apenas para os 'Indefinidos'
df_leads_compras.loc[df_leads_compras['Categoria'] == 'Indefinido', 'Categoria'] = \
    df_leads_compras['Mensagem'].apply(categorize)

In [32]:
df_leads_compras['status_agenda'].value_counts()

status_agenda
Não está na agenda    13220
Falta                   980
Atendido                497
Agendado                356
Cancelado               317
Name: count, dtype: int64

In [33]:
# Extra categories
df_leads_compras.loc[(df_leads_compras['Fonte'] == 'Indique e Multiplique') & (df_leads_compras['Categoria'] == 'Indefinido'), 'Categoria'] = 'Cortesia Indique'
df_leads_compras.loc[(df_leads_compras['Fonte'] == 'CRM BÔNUS') & (df_leads_compras['Categoria'] == 'Indefinido'), 'Categoria'] = 'Cortesia CRM Bônus'
df_leads_compras.loc[(df_leads_compras['Mensagem'] == 'Lead Pop Up de Saída. Ganhou Peeling Diamante.') & (df_leads_compras['Categoria'] == 'Indefinido'), 'Categoria'] = 'Cortesia PopUpSaida Peeling_D'
df_leads_compras.loc[(df_leads_compras['Mensagem'] == 'Lead Pop Up de Saída. Ganhou Massagem Modeladora.') & (df_leads_compras['Categoria'] == 'Indefinido'), 'Categoria'] = 'Cortesia PopUpSaida Mass_Mod'
df_leads_compras.loc[(df_leads_compras['Mensagem'] == 'Lead salvo pelo modal de WhatsApp da Isa') & (df_leads_compras['Categoria'] == 'Indefinido'), 'Categoria'] = 'Quer Falar no Whatsapp'

df_leads_compras['Categoria'].value_counts()

# indefinidos = df_leads_compras.loc[df_leads_compras['Categoria'] == 'Indefinido']
# indefinidos['Mensagem'].value_counts()

Categoria
Indefinido                       6205
Lavieen                          2031
Preenchimento                    1647
Enzimas                          1241
Ultraformer                      1159
Botox                             820
Bioestimulador                    724
Cortesia Indique                  471
Quer Falar no Whatsapp            241
Cortesia CRM Bônus                212
Institucional                     163
Cortesia PopUpSaida Peeling_D     147
Crio                              130
Cortesia PopUpSaida Mass_Mod      115
Limpeza                            64
Name: count, dtype: int64

In [34]:
df_leads_compras.columns

Index(['ID do lead', 'Email do lead', 'Telefone do lead', 'Mensagem',
       'Unidade_x', 'Fonte', 'Dia da entrada', 'Source', 'Medium', 'Term',
       'Content', 'Campaign', 'unidade_agenda', 'status_agenda',
       'data_na_agenda', 'procedimento', 'Telefones Limpos',
       'Telefone(s) do cliente', 'ID orçamento', 'Data venda', 'Unidade_y',
       'Valor líquido', 'qtd_orcamentos', 'valor_total_cliente', 'Categoria'],
      dtype='object')

In [35]:
colunas_para_usar = [
       # lead
       'ID do lead', 'Email do lead', 'Telefone do lead', 'Mensagem', 
       'Unidade_x', 'Fonte', 'Dia da entrada', 'Source', 'Categoria', 'Content', 'Campaign',
       
       # appointment
       'unidade_agenda', 'status_agenda', 'procedimento', 'data_na_agenda', 
       
       # sales
       'ID orçamento', 'Unidade_y', 'Data venda', 
       'Valor líquido', 'qtd_orcamentos', 'valor_total_cliente',
]

df_leads_compras_final = df_leads_compras[colunas_para_usar]

# stores_to_remove = ['HOMA', 'PLÁSTICA']
# df_leads_compras_final = df_leads_compras_final[~df_leads_compras_final['Unidade_x'].isin(stores_to_remove)]

In [36]:
# sort by valor líquido
df_leads_compras_final['comprou?'] = df_leads_compras_final['Valor líquido'].apply(lambda x: 'Comprou' if x > 0 else 'Não comprou')

/var/folders/c4/t_n58l316yl308hxjf5jlwsh0000gn/T/ipykernel_71094/1955182943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_leads_compras_final['comprou?'] = df_leads_compras_final['Valor líquido'].apply(lambda x: 'Comprou' if x > 0 else 'Não comprou')


In [37]:
df_leads_compras_final.columns

Index(['ID do lead', 'Email do lead', 'Telefone do lead', 'Mensagem',
       'Unidade_x', 'Fonte', 'Dia da entrada', 'Source', 'Categoria',
       'Content', 'Campaign', 'unidade_agenda', 'status_agenda',
       'procedimento', 'data_na_agenda', 'ID orçamento', 'Unidade_y',
       'Data venda', 'Valor líquido', 'qtd_orcamentos', 'valor_total_cliente',
       'comprou?'],
      dtype='object')

### df_mkt

In [38]:
# DESIRED COLUMNS:
df_leads_compras_final.columns = [
       'lead_id', 'lead_email', 'lead_phone', 'lead_message', 'lead_store', 'lead_source', 'lead_created_date',
       'lead_mkt_source', 'lead_mkt_category', 'lead_content', 'lead_campaign',
       
       'appointment_store', 'appointment_status', 'appointment_procedure', 'appointment_date', 

       'sales_id', 'sales_store', 'sales_date', 'sales_1st_value', 'sales_id_count', 'sales_tt_value', 'sales_buyer'
]

In [51]:
df_leads_compras_final['lead_source'].value_counts()

lead_source
Facebook Leads           11967
Google Pesquisa           1533
Indique e Multiplique      471
Mídia Whatsapp             385
Acesso Direto ao Site      333
CRM BÔNUS                  212
Instagram                  210
Busca Orgânica             193
Facebook postlink           49
Parceria                     6
Tiktok                       4
Google Meu Negócio           3
Blog Pró-Corpo               2
Indicação                    2
Name: count, dtype: int64

In [52]:
df_leads_compras_final['appointment_status'].value_counts()

# """
# Não está na agenda    11982
# Falta                  5376
# Atendido               1578
# Cancelado               965
# """

appointment_status
Não está na agenda    13220
Falta                   980
Atendido                497
Agendado                356
Cancelado               317
Name: count, dtype: int64

In [53]:
df_leads_compras_final['lead_store'].value_counts()

lead_store
PLÁSTICA        3268
HOMA            2676
JARDINS         1230
IPIRANGA         564
SANTOS           563
TATUAPÉ          520
COPACABANA       517
LONDRINA         485
OSASCO           464
CAMPINAS         419
ALPHAVILLE       403
LAPA             402
CENTRAL          395
MOEMA            394
SANTO AMARO      381
SOROCABA         380
TUCURUVI         355
MOOCA            347
TIJUCA           341
PRAIA GRANDE     317
SÃO BERNARDO     299
ITAIM            271
GUARULHOS        212
VILA MASCOTE     167
Name: count, dtype: int64

In [56]:
# Create a completely new DataFrame with all values converted to strings first
# This will eliminate any datetime/timezone complexities
df_str = pd.DataFrame()
for col in df_leads_compras_final.columns:
    df_str[col] = df_leads_compras_final[col].astype(str)

# Export the string version to Excel
df_str.to_excel('df_mkt.xlsx', index=False)

In [ ]:
# Ensure columns are numeric
df_leads_compras_final['sales_1st_value'] = df_leads_compras_final['sales_1st_value'].astype(str).str.replace('.', ',')
df_leads_compras_final['sales_tt_value'] = df_leads_compras_final['sales_tt_value'].astype(str).str.replace('.', ',')
df_leads_compras_final.fillna(0)

In [57]:
df_leads_compras_final

,lead_id,lead_email,lead_phone,lead_message,lead_store,lead_source,lead_created_date,lead_mkt_source,lead_mkt_category,lead_content,...,appointment_procedure,appointment_date,sales_id,sales_store,sales_date,sales_1st_value,sales_id_count,sales_tt_value,sales_buyer,count_of_phone
0,1450489,fcfrenkiel@gmail.com,13996101919,Cliente tem interesse no laser Lavieen (melasm...,SANTOS,Facebook Leads,2025-03-01 00:00:41,NaN,Lavieen,nan,...,NaN,None,435375.0,SANTOS,2025-03-03 16:18:37,"199,0",1.0,"199,0",Comprou,1
1,1450490,jujhuly1994@gmail.com,11953963771,Cliente tem interesse em Mamoplastia Redutora ...,PLÁSTICA,Facebook Leads,2025-03-01 00:01:54,NaN,Mamoplastia,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
2,1450491,edna01barbosa@gmail.com,3189469364,Interesse em Enzimas,COPACABANA,Facebook Leads,2025-03-01 00:02:05,NaN,Enzimas,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
3,1450492,neuzalimasilva100@gmail.com,11943030681,Lead Pop Up de Saída. Ganhou Peeling Diamante.,SANTO AMARO,Busca Orgânica,2025-03-01 00:04:39,NaN,Cortesia PopUpSaida Peeling_D,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
4,1450493,robertabianka18@gmail.com,11954516449,Cliente tem interesse em abdominoplastia,HOMA,Facebook Leads,2025-03-01 00:08:20,NaN,Abdominoplastia,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15365,1465967,marialesk.meta@gmail.com,11959837606,Cliente tem interesse em bioestimulador (Sculp...,SANTO AMARO,Facebook Leads,2025-03-19 23:52:37,NaN,Bioestimulador,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
15366,1465968,carol.espindola@hotmail.com.br,13992146045,Preenchimento Labial,SANTOS,Google Pesquisa,2025-03-19 23:53:06,adwordssantos,Preenchimento,PreenchimentoLabial,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
15367,1465969,doceria.dajessica2024@gmail.com,11986658281,Cliente tem interesse em laser Lavieen (Mancha...,ALPHAVILLE,Facebook Leads,2025-03-19 23:53:17,NaN,Lavieen,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
15368,1465970,marimar.r@hotmail.com,21964090027,Cliente tem interesse no laser Lavieen (melasm...,COPACABANA,Facebook Leads,2025-03-19 23:55:53,NaN,Lavieen,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1


In [44]:
# Adding the amount of times the phones of leads exist, flag to check duplicate values

df_leads_compras_final['count_of_phone'] = df_leads_compras_final.groupby('lead_phone')['lead_phone'].transform('count')


/var/folders/c4/t_n58l316yl308hxjf5jlwsh0000gn/T/ipykernel_71094/3136537312.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_leads_compras_final['count_of_phone'] = df_leads_compras_final.groupby('lead_phone')['lead_phone'].transform('count')


### Categories HOMA / PL

In [45]:
homa = df_leads_compras_final.loc[df_leads_compras_final['lead_store'] == 'HOMA']
plastica = df_leads_compras_final.loc[df_leads_compras_final['lead_store'] == 'PLÁSTICA']

In [46]:
# Dictionary mapping for surgery categories
surgery_category_mapping = {
    'Lipoaspiração': 'Lipoaspiração',
    'Abdominoplastia': 'Abdominoplastia',
    'Mastopexia': 'Mastopexia',
    'Prótese de Mama': 'Prótese de Mama',
    'Mamoplastia Redutora': 'Mamoplastia',
    'Mamoplastia': 'Mamoplastia'
}

# Ensure lead_message is string type
homa['lead_message'] = homa['lead_message'].astype(str)

# Simple function to categorize based on keywords
def categorize_surgery(text):
    for keyword, category in surgery_category_mapping.items():
        if pd.notna(text) and keyword.lower() in text.lower():
            return category
    return 'Indefinido'

# Apply categorization function to lead_message column
homa['lead_mkt_category'] = homa['lead_message'].apply(categorize_surgery)

/var/folders/c4/t_n58l316yl308hxjf5jlwsh0000gn/T/ipykernel_71094/3659401434.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  homa['lead_message'] = homa['lead_message'].astype(str)
/var/folders/c4/t_n58l316yl308hxjf5jlwsh0000gn/T/ipykernel_71094/3659401434.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  homa['lead_mkt_category'] = homa['lead_message'].apply(categorize_surgery)


In [47]:
# Dictionary mapping for surgery categories
surgery_category_mapping = {
    'Lipoaspiração': 'Lipoaspiração',
    'Abdominoplastia': 'Abdominoplastia',
    'Mastopexia': 'Mastopexia',
    'Prótese de Mama': 'Prótese de Mama',
    'Mamoplastia Redutora': 'Mamoplastia',
    'Mamoplastia': 'Mamoplastia'
}

# Ensure lead_message is string type
plastica['lead_message'] = plastica['lead_message'].astype(str)

# Simple function to categorize based on keywords
def categorize_surgery(text):
    for keyword, category in surgery_category_mapping.items():
        if pd.notna(text) and keyword.lower() in text.lower():
            return category
    return 'Indefinido'

# Apply categorization function to lead_message column
plastica['lead_mkt_category'] = plastica['lead_message'].apply(categorize_surgery)

/var/folders/c4/t_n58l316yl308hxjf5jlwsh0000gn/T/ipykernel_71094/120391562.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plastica['lead_message'] = plastica['lead_message'].astype(str)
/var/folders/c4/t_n58l316yl308hxjf5jlwsh0000gn/T/ipykernel_71094/120391562.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plastica['lead_mkt_category'] = plastica['lead_message'].apply(categorize_surgery)


In [48]:
# homa.to_excel('df_homa.xlsx', index=False)
# plastica.to_excel('df_plastica.xlsx', index=False)

In [49]:
# Dictionary mapping for surgery categories
surgery_category_mapping = {
    'Lipoaspiração': 'Lipoaspiração',
    'Abdominoplastia': 'Abdominoplastia',
    'Mastopexia': 'Mastopexia',
    'Prótese de Mama': 'Prótese de Mama',
    'Mamoplastia Redutora': 'Mamoplastia',
    'Mamoplastia': 'Mamoplastia'
}

# Ensure lead_message is string type
df_leads_compras_final['lead_message'] = df_leads_compras_final['lead_message'].astype(str)

# Simple function to categorize based on keywords
def categorize_surgery(text):
    for keyword, category in surgery_category_mapping.items():
        if pd.notna(text) and keyword.lower() in text.lower():
            return category
    return 'Indefinido'

# ONLY apply the categorization function to rows where lead_mkt_category is 'Indefinido'
df_leads_compras_final.loc[df_leads_compras_final['lead_mkt_category'] == 'Indefinido', 'lead_mkt_category'] = \
    df_leads_compras_final.loc[df_leads_compras_final['lead_mkt_category'] == 'Indefinido', 'lead_message'].apply(categorize_surgery)

df_leads_compras_final

/var/folders/c4/t_n58l316yl308hxjf5jlwsh0000gn/T/ipykernel_71094/2035621470.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_leads_compras_final['lead_message'] = df_leads_compras_final['lead_message'].astype(str)


,lead_id,lead_email,lead_phone,lead_message,lead_store,lead_source,lead_created_date,lead_mkt_source,lead_mkt_category,lead_content,...,appointment_procedure,appointment_date,sales_id,sales_store,sales_date,sales_1st_value,sales_id_count,sales_tt_value,sales_buyer,count_of_phone
0,1450489,fcfrenkiel@gmail.com,13996101919,Cliente tem interesse no laser Lavieen (melasm...,SANTOS,Facebook Leads,2025-03-01 00:00:41,NaN,Lavieen,nan,...,NaN,None,435375.0,SANTOS,2025-03-03 16:18:37,"199,0",1.0,"199,0",Comprou,1
1,1450490,jujhuly1994@gmail.com,11953963771,Cliente tem interesse em Mamoplastia Redutora ...,PLÁSTICA,Facebook Leads,2025-03-01 00:01:54,NaN,Mamoplastia,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
2,1450491,edna01barbosa@gmail.com,3189469364,Interesse em Enzimas,COPACABANA,Facebook Leads,2025-03-01 00:02:05,NaN,Enzimas,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
3,1450492,neuzalimasilva100@gmail.com,11943030681,Lead Pop Up de Saída. Ganhou Peeling Diamante.,SANTO AMARO,Busca Orgânica,2025-03-01 00:04:39,NaN,Cortesia PopUpSaida Peeling_D,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
4,1450493,robertabianka18@gmail.com,11954516449,Cliente tem interesse em abdominoplastia,HOMA,Facebook Leads,2025-03-01 00:08:20,NaN,Abdominoplastia,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15365,1465967,marialesk.meta@gmail.com,11959837606,Cliente tem interesse em bioestimulador (Sculp...,SANTO AMARO,Facebook Leads,2025-03-19 23:52:37,NaN,Bioestimulador,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
15366,1465968,carol.espindola@hotmail.com.br,13992146045,Preenchimento Labial,SANTOS,Google Pesquisa,2025-03-19 23:53:06,adwordssantos,Preenchimento,PreenchimentoLabial,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
15367,1465969,doceria.dajessica2024@gmail.com,11986658281,Cliente tem interesse em laser Lavieen (Mancha...,ALPHAVILLE,Facebook Leads,2025-03-19 23:53:17,NaN,Lavieen,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1
15368,1465970,marimar.r@hotmail.com,21964090027,Cliente tem interesse no laser Lavieen (melasm...,COPACABANA,Facebook Leads,2025-03-19 23:55:53,NaN,Lavieen,nan,...,NaN,None,NaN,Não é lead,NaN,nan,NaN,nan,Não comprou,1


In [50]:
df_leads_compras_final['lead_mkt_category'].value_counts()


lead_mkt_category
Lavieen                          2031
Abdominoplastia                  1779
Preenchimento                    1647
Lipoaspiração                    1571
Enzimas                          1241
Indefinido                       1219
Ultraformer                      1159
Botox                             820
Bioestimulador                    724
Mastopexia                        639
Prótese de Mama                   575
Cortesia Indique                  471
Mamoplastia                       422
Quer Falar no Whatsapp            241
Cortesia CRM Bônus                212
Institucional                     163
Cortesia PopUpSaida Peeling_D     147
Crio                              130
Cortesia PopUpSaida Mass_Mod      115
Limpeza                            64
Name: count, dtype: int64

# konquista

In [154]:
mes = 2

# Caminho da pasta onde estão os arquivos
file_path = '/Users/luisfaria/Desktop/sEngineer/dash/db/contacts.csv'

df_konquista = pd.read_csv(file_path, dtype={'phone': str})

df_konquista['created_at'] = pd.to_datetime(df_konquista['created_at'], format='ISO8601', utc=True)
df_konquista['month'] = df_konquista['created_at'].dt.month

df_konquista_filtered = df_konquista[df_konquista['month'] == mes]
mkt_relationship_tags = ['Botox', 'Instagram', 'Preenchimento']
df_konquista_filtered = df_konquista_filtered[df_konquista_filtered['relationship_tag'].isin(mkt_relationship_tags)]

desired_columns = ['phone', 'created_at', 'relationship_tag',
       'store', 'region', 'month',

        # Lead data
        'is_lead', 'lead_id', 'lead_status', 'lead_created_at', 'store_lead',
        # Appointment data
        'is_appointment', 'appointment_status', 'appointment_created_at','store_appointment',
        # Bills
        'is_bill_charge', 'bill_charge_created_at', 'bill_charge_id', 'bill_charge_installments', 'bill_charge_total_amount',
]

df_konquista_final = df_konquista_filtered[desired_columns]

In [155]:
# Convertendo 'created_at' para datetime caso não esteja nesse formato
df_konquista_final['created_at'] = pd.to_datetime(df_konquista_final['created_at'])

# Extraindo o ano e preenchendo a coluna 'year'
df_konquista_final['year'] = df_konquista_final['created_at'].dt.year
df_konquista_final = df_konquista_final.loc[df_konquista_final['year'] == 2025]
df_konquista_final

/var/folders/c4/t_n58l316yl308hxjf5jlwsh0000gn/T/ipykernel_21365/3277967207.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_konquista_final['created_at'] = pd.to_datetime(df_konquista_final['created_at'])
/var/folders/c4/t_n58l316yl308hxjf5jlwsh0000gn/T/ipykernel_21365/3277967207.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_konquista_final['year'] = df_konquista_final['created_at'].dt.year


,phone,created_at,relationship_tag,store,region,month,is_lead,lead_id,lead_status,lead_created_at,...,is_appointment,appointment_status,appointment_created_at,store_appointment,is_bill_charge,bill_charge_created_at,bill_charge_id,bill_charge_installments,bill_charge_total_amount,year
852,11977327998,2025-02-24 03:00:00+00:00,Instagram,CENTRAL,São Paulo,2,True,1456070.0,Não Conseguiu Entrar Em Contato,2025-03-08 14:50:50+00,...,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,2025
861,11941118036,2025-02-24 03:00:00+00:00,Instagram,CENTRAL,São Paulo,2,True,1447186.0,Agendado,2025-02-24 10:57:06+00,...,True,Reagendado,2025-02-25 21:30:00+00,MOEMA,False,NaN,NaN,NaN,NaN,2025
895,11930135947,2025-02-24 03:00:00+00:00,Instagram,CENTRAL,São Paulo,2,False,NaN,NaN,NaN,...,True,Atendido,2025-02-25 11:35:00+00,JARDINS,False,NaN,NaN,NaN,NaN,2025
957,11944594122,2025-02-24 03:00:00+00:00,Instagram,CENTRAL,São Paulo,2,False,NaN,NaN,NaN,...,True,Falta,2025-02-25 20:45:00+00,GUARULHOS,False,NaN,NaN,NaN,NaN,2025
1002,11971929868,2025-02-24 03:00:00+00:00,Instagram,CENTRAL,São Paulo,2,False,NaN,NaN,NaN,...,True,Cancelado,2025-02-27 20:15:00+00,VILA MASCOTE,False,NaN,NaN,NaN,NaN,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28550,12997155955,2025-02-11 03:00:00+00:00,Botox,CENTRAL,São Paulo,2,True,1436827.0,Não Conseguiu Entrar Em Contato,2025-02-15 13:48:35+00,...,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,2025
28551,11941989702,2025-02-10 03:00:00+00:00,Instagram,CENTRAL,São Paulo,2,False,NaN,NaN,NaN,...,True,Atendido,2025-02-12 16:45:00+00,SANTO AMARO,False,NaN,NaN,NaN,NaN,2025
28552,11967250611,2025-02-10 03:00:00+00:00,Instagram,CENTRAL,São Paulo,2,True,1435733.0,Solicitou Retorno,2025-02-14 15:51:24+00,...,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,2025
28553,11991817971,2025-02-07 03:00:00+00:00,Instagram,CENTRAL,São Paulo,2,True,1433421.0,Não Conseguiu Entrar Em Contato,2025-02-12 14:11:16+00,...,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,2025


In [156]:
# Define the initial values for the last ID and email increment
last_id = df_leads['ID do lead'].max() if not df_leads.empty else 0
email_base = "sem_email@email"  # Base for the fake email

# Create a new DataFrame with the mapped structure
df_new_leads = pd.DataFrame({
    'ID do lead': [last_id + i + 1 for i in range(len(df_konquista_filtered))],
    'Nome do lead': 'na',
    'Email do lead': [f"{email_base}{last_id + i + 1}.com" for i in range(len(df_konquista_filtered))],
    'Telefone do lead': df_konquista_filtered['phone'],
    'Mensagem': df_konquista_filtered['relationship_tag'],
    'Região': df_konquista_filtered['region'],
    'Unidade': df_konquista_filtered['store'],
    'Fonte': 'konquista',
    'Dia da entrada': df_konquista_filtered['created_at'],
    'Status': 'contato whatsapp',
    'Grupo de procedimento': df_konquista_filtered['relationship_tag'],
    'Quantidade de atendimentos': 'na',
    'GCLID': 'na',
    'Source': 'na',
    'Medium': 'na',
    'Term': 'na',
    'Content': 'na',
    'Campaign': 'na',
    'Site referência': 'na',
    'Termo de busca': 'na',
    'Dispositivo': 'na',
    'Nome da página': 'na',
    'Última atendente': 'na',
    'Último atendimento': 'na'
})

# Append the new DataFrame to the existing df_leads
df_leads_updated = pd.concat([df_leads, df_new_leads], ignore_index=True)

In [157]:
df_leads = df_leads_updated
df_leads

,ID do lead,Nome do lead,Email do lead,Telefone do lead,Mensagem,Região,Unidade,Fonte,Dia da entrada,Status,...,Medium,Term,Content,Campaign,Site referência,Termo de busca,Dispositivo,Nome da página,Última atendente,Último atendimento
0,1422278,Li Arantes,liliarantes4.2@gmail.com,11991488470,Cliente tem interesse em Mastopexia para corri...,São Paulo,HOMA,Facebook Leads,2025-02-01 00:01:35,Não Conseguiu Entrar Em Contato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dailany Lima Nascimento,2025-02-01 08:46:22
1,1422279,Andrea Msilva,andreia_548@hotmail.com,11974595985,Cliente tem interesse em preenchimento (lábios...,São Paulo,OSASCO,Facebook Leads,2025-02-01 00:02:35,Não Conseguiu Entrar Em Contato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Camilly Giselli Barros e Barros,2025-03-05 16:16:02
2,1422280,Layza Dias,lay_h_andrade@outlook.com,11983993269,Cliente tem interesse em laser Lavieen (melasm...,São Paulo,SANTO AMARO,Facebook Leads,2025-02-01 00:08:48,Não Conseguiu Entrar Em Contato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ketlyn Aparecida Ferreira Monteiro,2025-02-25 16:58:57
3,1422281,Regiane Silva Lima,regianesil.lima15@gmail.com,11940772079,Plástica abdominal,São Paulo,PLÁSTICA,Google Pesquisa,2025-02-01 00:09:17,Rejeitou,...,search,NaN,abdm_lookalikemeta,pmax,https://syndicatedsearch.goog/,/mais-cirurgia-plastica/abdominoplastia,Mobile,Abdominoplastia,Nicole Gabrielli Neves de Melo,2025-02-01 12:57:29
4,1422282,Silvana Alves,sileval@hotmail.com,11969482485,Cliente tem interesse em Mamoplastia Redutora ...,São Paulo,HOMA,Facebook Leads,2025-02-01 00:17:31,Não Conseguiu Entrar Em Contato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dailany Lima Nascimento,2025-02-01 08:43:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30490,1452770,na,sem_email@email1452770.com,12997155955,Botox,São Paulo,CENTRAL,konquista,2025-02-11 03:00:00+00:00,contato whatsapp,...,na,na,na,na,na,na,na,na,na,na
30491,1452771,na,sem_email@email1452771.com,11941989702,Instagram,São Paulo,CENTRAL,konquista,2025-02-10 03:00:00+00:00,contato whatsapp,...,na,na,na,na,na,na,na,na,na,na
30492,1452772,na,sem_email@email1452772.com,11967250611,Instagram,São Paulo,CENTRAL,konquista,2025-02-10 03:00:00+00:00,contato whatsapp,...,na,na,na,na,na,na,na,na,na,na
30493,1452773,na,sem_email@email1452773.com,11991817971,Instagram,São Paulo,CENTRAL,konquista,2025-02-07 03:00:00+00:00,contato whatsapp,...,na,na,na,na,na,na,na,na,na,na


### Download Excel

In [158]:
df_leads_compras_final.to_excel("df_mkt_compras_final.xlsx", index=False)

from google.colab import files

# Baixar o arquivo CSV para o seu computador
files.download('df_mkt_compras_final.xlsx')

ModuleNotFoundError: No module named 'google.colab'

# Rodar Script - Cirurgias

### Atualiza bases

In [1]:
folder_path = '/content/'
files = os.listdir(folder_path)
df_agd = None
df_leads = None
df_vendas = None
import os

for file_name in files:
    file_path = os.path.join(folder_path, file_name)

    # Verificar se é um arquivo Excel
    if file_name.endswith('.xlsx'):
        if 'agendamentos' in file_name.lower():
            df_agd = pd.read_excel(file_path)
            print(f"Arquivo de agendamentos carregado: {file_name}")
        elif 'leads' in file_name.lower():
            df_leads = pd.read_excel(file_path)
            print(f"Arquivo de leads carregado: {file_name}")
        elif 'venda' in file_name.lower():
            df_vendas = pd.read_excel(file_path)
            print(f"Arquivo de vendas carregado: {file_name}")
# Verificar se todos os DataFrames foram carregados
if df_agd is None:
    print("Nenhum arquivo de agendamentos encontrado.")
if df_leads is None:
    print("Nenhum arquivo de leads encontrado.")
if df_vendas is None:
    print("Nenhum arquivo de vendas encontrado.")

NameError: name 'os' is not defined

# Play

### Leads Atendidos

In [ ]:
unidades = ['HOMA', 'PLÁSTICA']
df_agd = df_agd.loc[df_agd['Unidade do agendamento'].isin(unidades)]

# Classificando os agendamentos
df_agd["proced_avaliação"] = df_agd['Procedimento'].str.contains(r'.*AVALIAÇÃO.*', case=False)
df_agd["agendamento"] = df_agd['Status'].str.contains(r'.*Atendido|Falta.*', case=False)
df_agd["comparecimento"] = df_agd['Status'].str.contains(r'.*Atendido.*', case=False)
df_agd["esta agendado?"] = df_agd['Status'].str.contains("Agendado", case=False)
df_agd["falta ou cancelado?"] = df_agd['Status'].str.contains(r'.*Falta|Cancelado.*', case=False)

# Criando dataframe com colunas que declaram: procedimento de avaliação = "AVALIAÇÃO" e status "Agendado" = "Atendido + Falta"
df_agendamentos_aval = df_agd[df_agd['proced_avaliação'] == True]
df_agendamentos_aval = df_agendamentos_aval[df_agendamentos_aval['agendamento'] == True]

# Criando dataframe com coluna adicional que declara status apenas "Atendido" pra puxar os comparecimentos
df_comparecimentos_aval = df_agd[df_agd['proced_avaliação'] == True]
df_comparecimentos_aval = df_comparecimentos_aval[df_comparecimentos_aval['agendamento'] == True]
df_comparecimentos_aval = df_comparecimentos_aval[df_comparecimentos_aval['comparecimento'] == True]

# Filtrando procedimentos!
procedimentos_que_vamos_olhar = [
                                #'RETORNO DE AVALIAÇÃO (ENTREGA EXAMES)',
                                'AVALIAÇÃO MAMOPLASTIA',
                                'AVALIAÇÃO PLÁSTICA DE ABDÔMEN',
                                'AVALIAÇÃO MAMOPLASTIA COM PRÓTESE',
                                'AVALIAÇÃO LIPOASPIRAÇÃO',
                                'AVALIAÇÃO PRÓTESE DE MAMA',
                                #'RETORNO CIRURGIA PLÁSTICA',
                                'AVALIAÇÃO RINOPLASTIA',
                                'AVALIAÇÃO BLEFAROPLASTIA',
                                'AVALIAÇÃO RITIDOPLASTIA',
                                'AVALIAÇÃO CIRURGIA ÍNTIMA',
                                'SEGUNDA OPINIÃO (AVALIAÇÃO CIRURGIA)'
]

# Usando loc para filtrar apenas "Procedimento" que estiver na minha lista:
df_agendamentos_aval = df_agendamentos_aval.loc[df_agendamentos_aval['Procedimento'].isin(procedimentos_que_vamos_olhar)]
df_comparecimentos_aval = df_comparecimentos_aval.loc[df_comparecimentos_aval['Procedimento'].isin(procedimentos_que_vamos_olhar)]

In [ ]:
df_leads

,ID do lead,Nome do lead,Email do lead,Telefone do lead,Mensagem,Região,Unidade,Fonte,Dia da entrada,Status,...,Medium,Term,Content,Campaign,Site referência,Termo de busca,Dispositivo,Nome da página,Última atendente,Último atendimento
0,1422278,Li Arantes,liliarantes4.2@gmail.com,11991488470,Cliente tem interesse em Mastopexia para corri...,São Paulo,HOMA,Facebook Leads,2025-02-01 00:01:35,Não Conseguiu Entrar Em Contato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dailany Lima Nascimento,2025-02-01 08:46:22
1,1422279,Andrea Msilva,andreia_548@hotmail.com,11974595985,Cliente tem interesse em preenchimento (lábios...,São Paulo,OSASCO,Facebook Leads,2025-02-01 00:02:35,Não Conseguiu Entrar Em Contato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vitoria Almeida Silva,2025-02-04 18:26:10
2,1422280,Layza Dias,lay_h_andrade@outlook.com,11983993269,Cliente tem interesse em laser Lavieen (melasm...,São Paulo,SANTO AMARO,Facebook Leads,2025-02-01 00:08:48,Não Conseguiu Entrar Em Contato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Julia Santos de Almeida,2025-02-12 18:30:18
3,1422281,Regiane Silva Lima,regianesil.lima15@gmail.com,11940772079,Plástica abdominal,São Paulo,PLÁSTICA,Google Pesquisa,2025-02-01 00:09:17,Rejeitou,...,search,NaN,abdm_lookalikemeta,pmax,https://syndicatedsearch.goog/,/mais-cirurgia-plastica/abdominoplastia,Mobile,Abdominoplastia,Nicole Gabrielli Neves de Melo,2025-02-01 12:57:29
4,1422282,Silvana Alves,sileval@hotmail.com,11969482485,Cliente tem interesse em Mamoplastia Redutora ...,São Paulo,HOMA,Facebook Leads,2025-02-01 00:17:31,Não Conseguiu Entrar Em Contato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dailany Lima Nascimento,2025-02-01 08:43:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19490,1441823,Ana Paula Castro,anapaula020870@gmail.com,11991988002,Cliente tem interesse em Mastopexia para corri...,São Paulo,PLÁSTICA,Facebook Leads,2025-02-19 23:55:08,Não Atendido,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19491,1441824,Liliane Moreira Santana Pimentel,lilianeyuri25@gmail.com,11967467822,Cliente tem interesse em Mastopexia para corri...,São Paulo,HOMA,Facebook Leads,2025-02-19 23:57:06,Não Atendido,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19492,1441825,Caroline G. Silva,caroline_703@hotmail.com,11976887414,Cliente tem interesse em Lipoaspiração | Segunda,São Paulo,PLÁSTICA,Facebook Leads,2025-02-19 23:57:55,Não Atendido,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19493,1441826,Vilmasantana Vilma,vilmavilmasantana829@gmail.com,13997394895,Interessado em sculptra,Santos,SANTOS,Facebook Leads,2025-02-19 23:58:40,Não Atendido,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Deixando só cirurgia
df_leads = df_leads.loc[df_leads['Unidade'].isin(unidades)]

# Função para limpar telefones
def clean_telephone(telefone):
    telefone = re.sub(r'[^\d]', '', str(telefone))  # Remove todos os caracteres não numéricos e garante que é string
    if telefone.startswith('55'):
        telefone = telefone[2:]  # Remove o prefixo +55
    return telefone

# Tratando os dados do df_leads
df_leads['Telefone do lead'] = df_leads['Telefone do lead'].fillna('Cliente sem telefone')
df_leads['Email do lead'] = df_leads['Email do lead'].fillna('None')
df_leads['Telefone do lead'] = df_leads['Telefone do lead'].astype(str)
df_leads['Telefone do lead'] = df_leads['Telefone do lead'].apply(clean_telephone)

# Limpar os telefones no df_agd, incluindo separar múltiplos telefones
df_comparecimentos_aval['Telefone'] = df_comparecimentos_aval['Telefone'].fillna('Cliente sem telefone')
df_comparecimentos_aval['Email'] = df_comparecimentos_aval['Email'].fillna('NA')
df_comparecimentos_aval['Telefone'] = df_comparecimentos_aval['Telefone'].astype(str)
df_comparecimentos_aval['Telefones Limpos'] = df_comparecimentos_aval['Telefone'].apply(lambda x: [clean_telephone(num) for num in str(x).split('/')])

# Criar sets de telefones e emails agendados para busca rápida
telefones_agendados = set(num for sublist in df_comparecimentos_aval['Telefones Limpos'] for num in sublist)
emails_agendados = set(df_comparecimentos_aval['Email'])

# Função para verificar se o lead foi agendado e retornar dados adicionais do df_agd
def eh_comparecimento(row):
    telefone = row['Telefone do lead']
    email = row['Email do lead']
    match = df_comparecimentos_aval[(df_comparecimentos_aval['Telefones Limpos'].apply(lambda x: telefone in x)) | (df_comparecimentos_aval['Email'] == email)]

    if not match.empty:
        data = match.iloc[0]['Data']
        proced_avaliação = match.iloc[0]['proced_avaliação']
        agendamento = match.iloc[0]['agendamento']
        comparecimento = match.iloc[0]['comparecimento']

        return pd.Series({
            'data_agenda': data,
            'proced_avaliação': proced_avaliação,
            'agendamento': agendamento,
            'comparecimento': comparecimento,
            'procedimento': match.iloc[0]['Procedimento'] # Novo. 19/12/2024 # RES: Sugestões de melhorias para o Self Service 2.5
        })

    return pd.Series({'data_agenda': None, 'proced_avaliação': None, 'agendamento': None, 'comparecimento': None})

# Aplicar a função no df_leads e expandir a saída
df_leads = df_leads.join(df_leads.apply(eh_comparecimento, axis=1))

In [ ]:
colunas_que_quero = ['ID do lead', 'Nome do lead', 'Email do lead', 'Telefone do lead', 'Mensagem', 'Unidade', 'Fonte', 'Dia da entrada',
                     'Status','Source', 'Medium', 'Term', 'Content', 'Campaign', 'proced_avaliação', 'agendamento','comparecimento', 'data_agenda', 'procedimento']

df_leads = df_leads[colunas_que_quero]

# substituindo os null values por "não compareceu"
df_leads['comparecimento'] = df_leads['comparecimento'].fillna('Não compareceu')

In [ ]:
# # Agrupando por Unidade e fonte a quantidade de ID do Lead
# # Reset the index to convert the Series to a DataFrame
# groupby_unidade_fonte = df_leads.groupby(['Unidade', 'Fonte','comparecimento'])['ID do lead'].nunique().reset_index(name='count')

# pivot_unidade_fonte = groupby_unidade_fonte.pivot_table(index=['Unidade','Fonte'],
#                                                         columns=['comparecimento'],
#                                                         values=['count'], # Use 'count' as the values column
#                                                         fill_value=0,

#                                                         )
# # Ensure "Compareceu" column exists
# if 'Compareceu' not in pivot_unidade_fonte.columns:
#     pivot_unidade_fonte['Compareceu'] = 0

# pivot_unidade_fonte.columns = ['Compareceu', 'Não Compareceu']

In [ ]:
# Agrupando por Unidade e fonte a quantidade de ID do Lead
# Reset the index to convert the Series to a DataFrame
groupby_unidade_fonte = df_leads.groupby(['Unidade', 'Fonte','comparecimento'])['ID do lead'].nunique().reset_index(name='count')

pivot_unidade_fonte = groupby_unidade_fonte.pivot_table(index=['Unidade','Fonte'],
                                                        columns=['comparecimento'],
                                                        values=['count'], # Use 'count' as the values column
                                                        fill_value=0,
                                                        )
# Ensure "Compareceu" column exists
if ('count', 'Compareceu') not in pivot_unidade_fonte.columns:
    pivot_unidade_fonte[('count', 'Compareceu')] = 0

# Flatten the MultiIndex columns for simplicity
pivot_unidade_fonte.columns = [col[1] for col in pivot_unidade_fonte.columns] #Extracting the second level of the MultiIndex
# Now you can assign the desired column names
# pivot_unidade_fonte.columns = ['Compareceu', 'Não Compareceu'] #This line is not needed anymore

In [ ]:
pivot_unidade_fonte

True  Não compareceu  Compareceu
Unidade  Fonte                                                  
HOMA     Acesso Direto ao Site   0.0            79.0           0
         Busca Orgânica          0.0            23.0           0
         Facebook Leads         20.0          3711.0           0
         Facebook postlink       0.0             1.0           0
         Google Pesquisa         0.0           708.0           0
         Instagram               0.0            30.0           0
PLÁSTICA Acesso Direto ao Site   1.0             8.0           0
         Busca Orgânica          0.0             1.0           0
         Facebook Leads         31.0          2700.0           0
         Google Pesquisa         3.0           170.0           0
         Instagram Leads         0.0            11.0           0
         Mídia Whatsapp          0.0             1.0           0

### Leads x Outros Status

In [ ]:
# separando leads que não compareceram para ver se estão agendados ou falta/cancelados
df_leads_que_nao_compareceram = df_leads.loc[df_leads['comparecimento'] == 'Não compareceu']

# Função para limpar telefones
def clean_telephone(telefone):
    telefone = re.sub(r'[^\d]', '', str(telefone))  # Remove todos os caracteres não numéricos e garante que é string
    if telefone.startswith('55'):
        telefone = telefone[2:]  # Remove o prefixo 55
    return telefone

# Limpar telefones no df_leads_que_nao_compareceram
df_leads_que_nao_compareceram['Telefone do lead'] = df_leads_que_nao_compareceram['Telefone do lead'].apply(clean_telephone)

# Limpar telefones no df_agd, incluindo separar múltiplos telefones
df_agd['Telefone'] = df_agd['Telefone'].fillna('Cliente sem telefone')
df_agd['Telefone'] = df_agd['Telefone'].astype(str)
df_agd['Telefones Limpos'] = df_agd['Telefone'].apply(lambda x: [clean_telephone(num) for num in str(x).split('/')])

# Verificação de status
df_agd["falta ou cancelado?"] = df_agd['Status'].str.contains(r'.*Falta|Cancelado.*', case=False, na=False)
df_agd["esta agendado?"] = df_agd['Status'].str.contains(r'(?<!Re)Agendado', case=False, na=False)


# procedimentos_que_vamos_olhar = ['AVALIAÇÃO TATUAGEM', 'AVALIAÇÃO INJETÁVEIS E INVASIVOS','AVALIAÇÃO ESTÉTICA', 'AVALIAÇÃO DEPILAÇÃO', 'AVALIAÇÃO MEDICINA ESTÉTICA']

# Filtrar os leads que estão agendados ou falta/cancelado
df_agd_filtrado = df_agd[(df_agd["esta agendado?"] == True) | (df_agd["falta ou cancelado?"] == True)]
df_agd_filtrado = df_agd_filtrado.loc[df_agd_filtrado['Procedimento'].isin(procedimentos_que_vamos_olhar)]

# Função para verificar se o lead está no df_agd_filtrado e trazer as informações adicionais
def verificar_status(row):
    telefone = row['Telefone do lead']
    email = row['Email do lead']
    match = df_agd_filtrado[(df_agd_filtrado['Telefones Limpos'].apply(lambda x: telefone in x)) | (df_agd_filtrado['Email'] == email)]

    if not match.empty:
        data = match.iloc[0]['Data']
        proced_avaliação = match.iloc[0]['proced_avaliação']
        agendamento = match.iloc[0]['esta agendado?']
        comparecimento = match.iloc[0]['falta ou cancelado?']
        status = match.iloc[0]['Status']
        procedimento = match.iloc[0]['Procedimento']

        return pd.Series({
            'data_agenda_novo': data,
            'proced_avaliação_novo': proced_avaliação,
            'agendamento_novo': agendamento,
            'comparecimento_novo': comparecimento,
            'status_novo': status,
            'procedimento_novo': procedimento
        })

    return pd.Series({
        'data_agenda_novo': None,
        'proced_avaliação_novo': None,
        'agendamento_novo': None,
        'comparecimento_novo': None,
        'status_novo': None,
        'procedimento_novo': None
    })

# Aplicar a função no df_leads_que_nao_compareceram e expandir a saída
df_leads_que_nao_compareceram = df_leads_que_nao_compareceram.join(df_leads_que_nao_compareceram.apply(verificar_status, axis=1))

In [ ]:
# Copiar o df_leads original
df_leads_copy = df_leads.copy()

# Selecionar os leads que não compareceram
df_leads_que_nao_compareceram2 = df_leads_copy.loc[df_leads_copy['comparecimento'] == 'Não compareceu']

# Atualizar a coluna 'comparecimento' com os valores de 'status_novo' em df_leads_que_nao_compareceram
df_leads_que_nao_compareceram.loc[:, 'comparecimento'] = df_leads_que_nao_compareceram['status_novo']

# Atualizar o DataFrame original
df_leads_copy.update(df_leads_que_nao_compareceram)

In [ ]:
df_leads_copy.loc[df_leads_copy['comparecimento'] == 'Não compareceu', 'comparecimento'] = 'Não está na agenda'
df_leads_copy.loc[df_leads_copy['comparecimento'] == True, 'comparecimento'] = 'Atendido'

### Categorias

In [ ]:
# Dicionário de mapeamento: chave é a palavra-chave, valor é a categoria
category_mapping = {
    'Abdominoplastia': 'Abdominoplastia',
    'Mamoplastia': 'Mamoplastia',
    'Mastopexia': 'Mamoplastia',
    'Prótese de Mama': 'Mamoplastia',
    'troca de prótese mamária': 'Mamoplastia',
    'Lipoaspiração': 'Lipoaspiração',
    'Lipoescultura': 'Lipoaspiração',
    'Lipo HD': 'Lipoaspiração',
    'Cirurgia Plástica': 'Cirurgia Plástica',
    'cirurgia': 'Cirurgia Plástica',
    'abdominoplastia': 'Abdominoplastia',
    'mamoplastia': 'Mamoplastia',
    'abdm': 'Abdominoplastia',
    'lipoaspiracao': 'Lipoaspiração',
    'lipo': 'Lipoaspiração',
}

# Transformar colunas ['Content'] e ['Mensagem'] em string
df_leads_copy['Content'] = df_leads_copy['Content'].astype(str)
df_leads_copy['Mensagem'] = df_leads_copy['Mensagem'].astype(str)

# Função para categorizar com base nas palavras-chave
def categorize(text):
    for keyword, category in category_mapping.items():
        if pd.notna(text) and keyword.lower() in text.lower():  # Verifica se a palavra-chave está presente
            return category
    return 'Indefinido'  # Categoria padrão se não encontrar correspondência

# Passo 1: Aplicar a função de categorização na coluna 'Content'
df_leads_copy['Categoria'] = df_leads_copy['Content'].apply(categorize)

# Passo 2: Aplicar a função de categorização na coluna 'Mensagem', apenas para os 'Indefinidos'
df_leads_copy.loc[df_leads_copy['Categoria'] == 'Indefinido', 'Categoria'] = \
    df_leads_copy['Mensagem'].apply(categorize)

### Colunas

In [ ]:
df_leads_copy.columns

Index(['ID do lead', 'Nome do lead', 'Email do lead', 'Telefone do lead',
       'Mensagem', 'Unidade', 'Fonte', 'Dia da entrada', 'Status', 'Source',
       'Medium', 'Term', 'Content', 'Campaign', 'proced_avaliação',
       'agendamento', 'comparecimento', 'data_agenda', 'procedimento',
       'Categoria'],
      dtype='object')

In [ ]:
# DESIRED COLUMNS:
df_leads_copy.drop(columns=['proced_avaliação', 'agendamento'], inplace=True)

df_leads_copy.columns = ['lead_id', 'lead_name', 'lead_email', 'lead_phone',
       'lead_message', 'lead_store', 'lead_source', 'lead_created_date', 'lead_status', 'utm_source',
       'utm_medium', 'utm_term', 'utm_content', 'utm_campaign', 'appointment_status',
       'appointment_date', 'appointment_procedure', 'lead_category']

# ADJUSTING ORDER:
df_leads_copy = df_leads_copy[['lead_id', 'lead_name', 'lead_email', 'lead_phone',
       'lead_message', 'lead_store', 'lead_source', 'lead_created_date', 'lead_status', 'utm_source',
       'utm_medium', 'utm_term', 'utm_content', 'utm_campaign', 'lead_category', 'appointment_status',
       'appointment_date', 'appointment_procedure']]

# Cola os dados na Planilha

In [ ]:
url_to_paste = 'https://docs.google.com/spreadsheets/d/1WlXkEgfv8DiFB-bM7CKqmSady_dA0GYCWG09wqzoxHc/edit'

# Opening spreadsheet method
spreadsheet = gc.open_by_url(url_to_paste)

# Selecting desired sheets
sheet_data = spreadsheet.worksheet('cirurgias')

# Colar o DataFrame na aba especificada
sheet_data.clear()
set_with_dataframe(sheet_data, df_leads_copy)

### Download Excel

In [ ]:
# salvando df_leads_copy em excel
df_leads_copy.to_excel("df_mkt_x_agenda_cirurgia.xlsx", index=False)

from google.colab import files

# Baixar o arquivo CSV para o seu computador
files.download('df_mkt_x_agenda_cirurgia.xlsx')
print("Tudo certo... baixando excel")

# Konquista

In [ ]:
# Selecione o mês
mes = 1

#### Bases 31/01/2025


In [ ]:
# # 31/01/2025

# import os
# import pandas as pd

# # Caminho da pasta onde estão os arquivos
# folder_path = '/content/'
# files = os.listdir(folder_path)

# # Inicializar estruturas para armazenamento
# df_leads = None
# agd_dfs = []  # Lista para armazenar DataFrames de agd
# vmb_dfs = []  # Lista para armazenar DataFrames de vmb

# # Processar cada arquivo na pasta
# for file_name in files:
#     file_path = os.path.join(folder_path, file_name)

#     # Processar apenas arquivos Excel
#     if file_name.lower().endswith('.xlsx'):
#         file_lower = file_name.lower()

#         # LEADS: Carregar único arquivo
#         if file_lower.startswith('leads'):
#             if df_leads is not None:
#                 print(f'Aviso: múltiplos arquivos de leads encontrados! Usando o último: {file_name}')
#             df_leads = pd.read_excel(file_path)
#             print(f'Arquivo de leads carregado: {file_name}')

#         # AGD: Adicionar à lista para concatenação
#         elif file_lower.startswith('agd'):
#             df = pd.read_excel(file_path)
#             agd_dfs.append(df)
#             print(f'Arquivo agd encontrado: {file_name}')

#         # VMB: Adicionar à lista para concatenação
#         elif file_lower.startswith('vmb'):
#             df = pd.read_excel(file_path)
#             vmb_dfs.append(df)
#             print(f'Arquivo vmb encontrado: {file_name}')

# # Concatenar os DataFrames
# df_agd = pd.concat(agd_dfs, ignore_index=True) if agd_dfs else None
# df_vendas = pd.concat(vmb_dfs, ignore_index=True) if vmb_dfs else None

# # Verificar resultados
# print('\nStatus do carregamento:')
# print(f'Leads: {"Carregado" if df_leads is not None else "Não encontrado"}')
# print(f'AGD: {len(agd_dfs)} arquivo(s) concatenado(s)' if agd_dfs else 'AGD: Não encontrado')
# print(f'VMB: {len(vmb_dfs)} arquivo(s) concatenado(s)' if vmb_dfs else 'VMB: Não encontrado')

Arquivo vmb encontrado: vmb_2025_01.xlsx
Arquivo de leads carregado: leads_2025_01.xlsx
Arquivo agd encontrado: agd_2025_01.xlsx

Status do carregamento:
Leads: Carregado
AGD: 1 arquivo(s) concatenado(s)
VMB: 1 arquivo(s) concatenado(s)


### Load data

In [ ]:
import pandas as pd

path = '/content/drive/MyDrive/_paid_media_reports/core_contact_rows.csv'

# Define the dtype for the phone column as string
df_konquista = pd.read_csv(path, dtype={'phone': str})

# Month column
# df_konquista['month'] = pd.to_datetime(df_konquista['created_at']).dt.month

<ipython-input-642-d1255ea88fb5>:6: DtypeWarning: Columns (18,25,31,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df_konquista = pd.read_csv(path, dtype={'phone': str})


In [ ]:
df_konquista['created_at'] = pd.to_datetime(df_konquista['created_at'], format='ISO8601', utc=True)
df_konquista['month'] = df_konquista['created_at'].dt.month

In [ ]:
# usando o mês selecionado
df_konquista_filtered = df_konquista[df_konquista['month'] == mes]
df_konquista_filtered.columns

Index(['id', 'name', 'phone', 'created_at', 'relationship_tag', 'source',
       'store', 'region', 'reference_code', 'external_tag', 'tag', 'status',
       'is_lead', 'lead_id', 'lead_status', 'lead_created_at',
       'lead_last_checked', 'lead_check_count', 'store_lead', 'is_appointment',
       'appointment_id', 'appointment_status', 'appointment_created_at',
       'appointment_last_checked', 'appointment_check_count',
       'store_appointment', 'user_id', 'botox_messages_sent',
       'last_message_sent_at', 'preenchimento_messages_sent',
       'bill_charge_check_count', 'bill_charge_created_at', 'bill_charge_id',
       'bill_charge_installments', 'bill_charge_last_checked',
       'bill_charge_status', 'bill_charge_total_amount', 'is_bill_charge',
       'store_bill_charge', 'bill_charge_total_history', 'month'],
      dtype='object')

In [ ]:
mkt_relationship_tags = ['Botox', 'Instagram', 'Preenchimento']
df_konquista_filtered = df_konquista_filtered[df_konquista_filtered['relationship_tag'].isin(mkt_relationship_tags)]

In [ ]:
desired_columns = ['phone', 'created_at', 'relationship_tag',
       'store', 'region', 'month',

        # Lead data
        'is_lead', 'lead_id', 'lead_status', 'lead_created_at', 'store_lead',
        # Appointment data
        'is_appointment', 'appointment_status', 'appointment_created_at','store_appointment',
        # Bills
        'is_bill_charge', 'bill_charge_created_at', 'bill_charge_id', 'bill_charge_installments', 'bill_charge_total_amount',
]

df_konquista_final = df_konquista_filtered[desired_columns]

In [ ]:
df_konquista_final

,phone,created_at,relationship_tag,store,region,month,is_lead,lead_id,lead_status,lead_created_at,store_lead,is_appointment,appointment_status,appointment_created_at,store_appointment,is_bill_charge,bill_charge_created_at,bill_charge_id,bill_charge_installments,bill_charge_total_amount
2380,21975259821,2025-01-11 03:00:00+00:00,Botox,CENTRAL,Rio de Janeiro,1,True,NaN,NaN,2025-01-23 13:00:46.069189+00,NaN,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
2406,11970882965,2025-01-11 03:00:00+00:00,Botox,CENTRAL,São Paulo,1,True,NaN,NaN,2025-01-23 12:59:26.576567+00,NaN,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
2418,11987156960,2025-01-11 03:00:00+00:00,Botox,CENTRAL,São Paulo,1,True,NaN,NaN,2025-01-23 12:50:09.841569+00,NaN,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
2419,11941073712,2025-01-11 03:00:00+00:00,Botox,CENTRAL,São Paulo,1,True,NaN,NaN,2025-01-23 12:54:08.392828+00,NaN,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
2435,11953696452,2025-01-11 03:00:00+00:00,Botox,CENTRAL,São Paulo,1,False,NaN,NaN,NaN,NaN,True,Agendado,2025-01-13 16:30:00+00,JARDINS,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65665,21976659515,2025-01-05 03:00:00+00:00,Botox,CENTRAL,Rio de Janeiro,1,True,NaN,NaN,2025-01-28 15:42:10.845529+00,NaN,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
65666,4399652852,2025-01-05 03:00:00+00:00,Botox,CENTRAL,Londrina,1,True,NaN,NaN,2025-01-28 15:58:52.278341+00,NaN,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
65667,11942009519,2025-01-05 03:00:00+00:00,Botox,CENTRAL,São Paulo,1,True,NaN,NaN,2025-01-28 16:57:18.673396+00,NaN,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
65813,4497712612,2025-01-28 02:56:12.271722+00:00,Instagram,CENTRAL,Londrina,1,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

# Define the initial values for the last ID and email increment
last_id = df_leads['ID do lead'].max() if not df_leads.empty else 0
email_base = "sem_email@email"  # Base for the fake email

# Create a new DataFrame with the mapped structure
df_new_leads = pd.DataFrame({
    'ID do lead': [last_id + i + 1 for i in range(len(df_konquista_filtered))],
    'Nome do lead': 'na',
    'Email do lead': [f"{email_base}{last_id + i + 1}.com" for i in range(len(df_konquista_filtered))],
    'Telefone do lead': df_konquista_filtered['phone'],
    'Mensagem': df_konquista_filtered['relationship_tag'],
    'Região': df_konquista_filtered['region'],
    'Unidade': df_konquista_filtered['store'],
    'Fonte': 'konquista',
    'Dia da entrada': df_konquista_filtered['created_at'],
    'Status': 'contato whatsapp',
    'Grupo de procedimento': df_konquista_filtered['relationship_tag'],
    'Quantidade de atendimentos': 'na',
    'GCLID': 'na',
    'Source': 'na',
    'Medium': 'na',
    'Term': 'na',
    'Content': 'na',
    'Campaign': 'na',
    'Site referência': 'na',
    'Termo de busca': 'na',
    'Dispositivo': 'na',
    'Nome da página': 'na',
    'Última atendente': 'na',
    'Último atendimento': 'na'
})

# Append the new DataFrame to the existing df_leads
df_leads_updated = pd.concat([df_leads, df_new_leads], ignore_index=True)

In [ ]:
df_leads = df_leads_updated
df_leads

,ID do lead,Nome do lead,Email do lead,Telefone do lead,Mensagem,Região,Unidade,Fonte,Dia da entrada,Status,...,Medium,Term,Content,Campaign,Site referência,Termo de busca,Dispositivo,Nome da página,Última atendente,Último atendimento
0,1388942,Darci DelMonaco,darcidms@uol.com.br,11992617596,Interessado Preenchimento,São Paulo,IPIRANGA,Facebook Leads,2025-01-01 00:06:52,Não Conseguiu Entrar Em Contato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sarah de jesus dos Santos Pilatos,2025-01-17 09:21:40
1,1388943,𝕷𝖔𝖓,marlonhiiigh@gmail.com,11914853157,Cliente tem interesse em laser Lavieen (melasm...,São Paulo,JARDINS,Facebook Leads,2025-01-01 00:09:07,Não Conseguiu Entrar Em Contato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ana Carolina Proenca Morais,2025-01-14 18:44:44
2,1388944,Antonia Gonçalves,goncalvesmenezesm@gmail.com,11965693279,Cliente tem interesse em preenchimento (lábios...,São Paulo,OSASCO,Facebook Leads,2025-01-01 00:10:35,Agendado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vitoria Almeida Silva,2025-01-02 11:43:00
3,1388945,Davidaloto,b.6568.00.6.0@gmail.com,88289542987,NaN,São Paulo,SANTO AMARO,Acesso Direto ao Site,2025-01-01 00:14:08,Telefone Errado,...,NaN,NaN,NaN,NaN,https://www.procorpoestetica.com.br/,NaN,Desktop,Pró-Corpo | Estética Avançada,Rayana Ketlyn Araujo de Almeida,2025-01-02 16:19:36
4,1388946,Viviane Rodrigues,vyvyrso@gmail.com,15998159345,Cliente tem interesse em abdominoplastia,São Paulo,HOMA,Facebook Leads,2025-01-01 00:21:23,Não Conseguiu Entrar Em Contato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bruna Baez dos Santos,2025-01-02 17:43:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35742,1424698,na,sem_email@email1424698.com,21976659515,Botox,Rio de Janeiro,CENTRAL,konquista,2025-01-05 03:00:00+00:00,contato whatsapp,...,na,na,na,na,na,na,na,na,na,na
35743,1424699,na,sem_email@email1424699.com,4399652852,Botox,Londrina,CENTRAL,konquista,2025-01-05 03:00:00+00:00,contato whatsapp,...,na,na,na,na,na,na,na,na,na,na
35744,1424700,na,sem_email@email1424700.com,11942009519,Botox,São Paulo,CENTRAL,konquista,2025-01-05 03:00:00+00:00,contato whatsapp,...,na,na,na,na,na,na,na,na,na,na
35745,1424701,na,sem_email@email1424701.com,4497712612,Instagram,Londrina,CENTRAL,konquista,2025-01-28 02:56:12.271722+00:00,contato whatsapp,...,na,na,na,na,na,na,na,na,na,na


### Global Budget

In [ ]:
# 31/01/2025

import os
import pandas as pd

# Caminho da pasta onde estão os arquivos
folder_path = '/content/'
files = os.listdir(folder_path)

vmb_dfs = []  # Lista para armazenar DataFrames de vmb

# Processar cada arquivo na pasta
for file_name in files:
    file_path = os.path.join(folder_path, file_name)

    # Processar apenas arquivos Excel
    if file_name.lower().endswith('.xlsx'):
        file_lower = file_name.lower()

        # VMB: Adicionar à lista para concatenação
        if file_lower.startswith('vmb'):
            df = pd.read_excel(file_path)
            vmb_dfs.append(df)
            print(f'Arquivo vmb encontrado: {file_name}')

# Concatenar os DataFrames
df_vendas = pd.concat(vmb_dfs, ignore_index=True) if vmb_dfs else None

# Verificar resultados
print('\nStatus do carregamento:')
print(f'VMB: {len(vmb_dfs)} arquivo(s) concatenado(s)' if vmb_dfs else 'VMB: Não encontrado')

Arquivo vmb encontrado: vmb_2024_05.xlsx
Arquivo vmb encontrado: vmb_2024_07.xlsx
Arquivo vmb encontrado: vmb_2024_06.xlsx
Arquivo vmb encontrado: vmb_2025_01.xlsx
Arquivo vmb encontrado: vmb_2024_08.xlsx
Arquivo vmb encontrado: vmb_2024_02.xlsx
Arquivo vmb encontrado: vmb_2024_03.xlsx
Arquivo vmb encontrado: vmb_2024_11.xlsx
Arquivo vmb encontrado: vmb_2024_10.xlsx
Arquivo vmb encontrado: vmb_2024_09.xlsx
Arquivo vmb encontrado: vmb_2024_12.xlsx
Arquivo vmb encontrado: vmb_2024_04.xlsx

Status do carregamento:
VMB: 12 arquivo(s) concatenado(s)


In [ ]:
df_vendas.columns

Index(['ID orçamento', 'Data orçamento', 'Status', 'Revenda', 'Data venda',
       'Mês venda', 'Hora venda', 'Unidade', 'Consultor', 'Avaliador',
       'Valor tabela', 'Valor desconto', 'Valor líquido', 'Grupo procedimento',
       'Procedimento', 'Quantidade', 'Valor tabela item',
       'Valor % desconto item', 'valor desconto item', 'Valor liquido item',
       'Fonte do cadastro do cliente', 'Nome cliente', 'CPF cliente',
       'ID cliente', 'Email do cliente', 'Telefone(s) do cliente',
       'Data nascimento cliente', 'Profissão cliente', 'Cortesia?'],
      dtype='object')

In [ ]:
# Phone Number, Total "Valor Líquido" Group by, First Purchase Day Orçamento ID, First Purchase Orçamento ID sum 'Valor líquido', Last Purchase Day Orçamento ID, Last Purchase Orçamento ID sum 'Valor líquido', Count Unique Orçamento ID, Average Orçamento Value ('Total "Valor Líquido" Group by'/ 'Count Unique Orçamento ID')

In [ ]:
df_vendas['Status'].value_counts()

,ID orçamento,Data orçamento,Status,Revenda,Data venda,Mês venda,Hora venda,Unidade,Consultor,Avaliador,...,Valor liquido item,Fonte do cadastro do cliente,Nome cliente,CPF cliente,ID cliente,Email do cliente,Telefone(s) do cliente,Data nascimento cliente,Profissão cliente,Cortesia?
0,346163,2023-12-27 15:12:11,Cancelado,NÃO,2024-05-14 16:33:39,Maio,16:33:39,BELO HORIZONTE,Natalia Vasconcelos Rios Ramos,NaN,...,2294.00,Google Pesquisa,Fernanda Machado Freitas,615.862.856-53,595984.0,fmfprojetos2010@gmail.com,31983509822,1968-09-16 00:00:00,Administrador (a),Sim
1,358164,2024-03-04 22:24:01,Finalizado,NÃO,2024-05-03 19:12:01,Maio,19:12:01,TIJUCA,Iasmin Primo Gandra,NaN,...,290.00,Facebook Leads,Jose Luciano Ribeiro da Silva,011.973.214-97,938528.0,lucianoribeirosilva919@gmail.com,21982633341,1980-12-06 00:00:00,Porteiro,Não
2,365913,2024-04-11 11:46:50,Finalizado,SIM,2024-05-24 11:22:49,Maio,11:22:49,CAMPINAS,Raynara Melo da Silva,ANA PAULA DE ALENCAR BENASSI ROCHA,...,1100.00,Google Pesquisa,Vanessa Fabiana Nallin Liiz,312.983.378-10,791640.0,vanessafnallin@hotmail.com,19992741709,1984-07-09 00:00:00,Gerente de RH,Não
3,366378,2024-04-13 09:19:54,Finalizado,NÃO,2024-05-21 13:57:41,Maio,13:57:41,BELO HORIZONTE,Larissa Ferraz El Malih,NaN,...,989.92,Blip,Natalia Pereira Ramos,098.619.816-16,673538.0,natally_ramos@hotmail.com.br,31984191989,1992-09-20 00:00:00,Outros,Não
4,366794,2024-04-15 20:25:43,Finalizado,NÃO,2024-05-10 11:49:56,Maio,11:49:56,ITAIM,Keylla Cristine Silva De Paula,NaN,...,90.00,CRM BÔNUS,Camila Nascimento Pianco Da Silva,101.089.919-89,950257.0,Camilapianco@gmail.com,11937798786,1995-11-06 00:00:00,Assistente,Sim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72570,371182,2024-04-30 22:17:40,Finalizado,SIM,2024-04-30 22:18:37,Abril,22:18:37,OSASCO,BKO VENDAS,Carla Lopes Custodio,...,NaN,NaN,Rodolfo Mendes Costa,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72571,371184,2024-04-30 22:17:40,Finalizado,SIM,2024-04-30 22:18:37,Abril,22:18:37,LONDRINA,Laryssa Benicio Santos,Arielly Cristina Brito Souto,...,NaN,NaN,Aparecida Fernandes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72572,371985,2024-04-30 22:17:40,Finalizado,SIM,2024-04-30 22:18:37,Abril,22:18:37,LONDRINA,Kauana Withoft,Arielly Cristina Brito Souto,...,NaN,NaN,Maria de Lourdes Oliveira,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72573,371991,2024-04-30 22:17:40,Finalizado,SIM,2024-04-30 22:18:37,Abril,22:18:37,TATUAPÉ,Thais Vieira Matias,Giulia de Oliveira Quintino,...,NaN,NaN,Shirley Aparecida Grota do Nascimento,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Step 1: Convert 'Data venda' to datetime
df_vendas['Data venda'] = pd.to_datetime(df_vendas['Data venda'])

# Step 2: Group by 'Nome cliente'
grouped = df_vendas.groupby('Nome cliente')

# Step 3: Calculate the required values

# Total 'Valor Líquido' Group by client name
total_valor_liquido = grouped['Valor líquido'].sum()

# First Purchase Day Orçamento ID and sum of 'Valor líquido' for the first purchase
first_purchase_day = grouped['Data venda'].min()
first_purchase_valor = grouped.apply(lambda x: x.loc[x['Data venda'] == x['Data venda'].min(), 'Valor líquido'].sum())

# Last Purchase Day Orçamento ID and sum of 'Valor líquido' for the last purchase
last_purchase_day = grouped['Data venda'].max()
last_purchase_valor = grouped.apply(lambda x: x.loc[x['Data venda'] == x['Data venda'].max(), 'Valor líquido'].sum())

# Count of Unique Orçamento IDs
unique_orcamento_count = grouped['ID orçamento'].nunique()

# Average 'Valor Líquido' per unique Orçamento ID
avg_orcamento_value = total_valor_liquido / unique_orcamento_count

# Step 4: Add Email and Phone Number - get the first occurrence of Email and Phone per client
client_info = grouped[['Email do cliente', 'Telefone(s) do cliente']].first()

# Step 5: Create the new DataFrame with the required columns
result2 = pd.DataFrame({
    'Cliente': total_valor_liquido.index,
    'Email': client_info['Email do cliente'].values,
    'Telefone': client_info['Telefone(s) do cliente'].values,
    'Total Valor Líquido Group by': total_valor_liquido.values,
    'First Purchase Day Orçamento ID': first_purchase_day.values,
    'First Purchase Orçamento ID sum Valor líquido': first_purchase_valor.values,
    'Last Purchase Day Orçamento ID': last_purchase_day.values,
    'Last Purchase Orçamento ID sum Valor líquido': last_purchase_valor.values,
    'Count Unique Orçamento ID': unique_orcamento_count.values,
    'Average Orçamento Value': avg_orcamento_value.values
})

<ipython-input-736-c2b71bb5d993>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  first_purchase_valor = grouped.apply(lambda x: x.loc[x['Data venda'] == x['Data venda'].min(), 'Valor líquido'].sum())
<ipython-input-736-c2b71bb5d993>:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_purchase_valor = grouped.apply(lambda x: x.loc[x['Data venda'] == x['Data venda'].max(), 'Valor líquido'].sum())


In [ ]:
result2

NameError: name 'result2' is not defined

In [ ]:
funil = '/content/funil 2.5.xlsx'
df_funil = pd.read_excel(funil, sheet_name='consolidado')

In [ ]:
df_funil_buyers = df_funil.loc[df_funil['sales_buyer'] == 'Comprou']
df_funil_buyers

,lead_id,lead_phone,lead_store,lead_font,lead_created_date,lead_category,appointment_status,appointment_date,appointment_procedure,sales_date,sales_store,sales_value,sales_buyer,count,sales_value_shared,sales_value_total_by_lead,sales_value_total_by_lead_shared,mes
15,1329012,11981630412,VILA MASCOTE,Facebook Leads,2024-11-01 00:25:10,Lavieen,Atendido,2024-11-07,AVALIAÇÃO ESTÉTICA,2024-11-07 19:17:39,VILA MASCOTE,597.99,Comprou,1,597.99,597,597.0,11
20,1329022,43991146611,LONDRINA,Google Pesquisa,2024-11-01 00:39:37,Cortesia PopUpSaida Peeling_D,Atendido,2024-11-07,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-07 12:48:02,LONDRINA,50.00,Comprou,1,50.00,50,50.0,11
45,1329070,11966299163,MOEMA,Facebook Leads,2024-11-01 03:02:14,Lavieen,Atendido,2024-11-07,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-28 19:14:40,MOEMA,6600.00,Comprou,1,6600.00,6900,6900.0,11
65,1329097,11986640457,JARDINS,Facebook Leads,2024-11-01 05:46:23,Ultraformer,Atendido,2024-11-01,AVALIAÇÃO ESTÉTICA,2024-11-01 13:08:13,VILA MASCOTE,2529.96,Comprou,1,2529.96,2529,2529.0,11
96,1329141,11947376615,SÃO BERNARDO,Google Pesquisa,2024-11-01 07:24:14,Lavieen,Atendido,2024-11-07,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-27 16:17:52,SÃO BERNARDO,597.00,Comprou,1,597.00,693,693.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214376,1424347,11944805721,CENTRAL,konquista,2025-01-16 03:00:00+00:00,Preenchimento,Atendido,2025-01-16,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2025-01-16 12:24:38,MOOCA,850.00,Comprou,2,425.00,850,425.0,1
214458,1424429,11911250170,CENTRAL,konquista,2025-01-21 03:00:00+00:00,Indefinido,Não está na agenda,NaT,NaN,2025-01-21 13:51:00,ITAIM,90.00,Comprou,1,90.00,90,90.0,1
214477,1424448,11986240435,CENTRAL,konquista,2025-01-21 03:00:00+00:00,Indefinido,Atendido,2025-01-23,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2025-01-23 18:46:11,LAPA,900.00,Comprou,1,900.00,900,900.0,1
214497,1424468,11981045454,CENTRAL,konquista,2025-01-21 03:00:00+00:00,Indefinido,Atendido,2025-01-23,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2025-01-23 14:00:51,SÃO BERNARDO,780.00,Comprou,1,780.00,780,780.0,1


In [ ]:
df_funil_non_buyers = df_funil.loc[~(df_funil['sales_buyer'] == 'Comprou')]
df_funil_non_buyers

,lead_id,lead_phone,lead_store,lead_font,lead_created_date,lead_category,appointment_status,appointment_date,appointment_procedure,sales_date,sales_store,sales_value,sales_buyer,count,sales_value_shared,sales_value_total_by_lead,sales_value_total_by_lead_shared,mes
0,1328991,21969168060,TIJUCA,Facebook Leads,2024-11-01 00:01:19,Ultraformer,Não está na agenda,NaT,NaN,-,Não é lead,NaN,Não comprou,1,0.0,0,0.0,11
1,1328993,11958819878,SÃO BERNARDO,Facebook Leads,2024-11-01 00:01:45,Preenchimento,Não está na agenda,NaT,NaN,-,Não é lead,NaN,Não comprou,1,0.0,0,0.0,11
2,1328995,43999713308,LONDRINA,Facebook Leads,2024-11-01 00:06:35,Lavieen,Não está na agenda,NaT,NaN,-,Não é lead,NaN,Não comprou,1,0.0,0,0.0,11
3,1328996,11985453795,SOROCABA,Facebook Leads,2024-11-01 00:07:31,Preenchimento,Não está na agenda,NaT,NaN,-,Não é lead,NaN,Não comprou,1,0.0,0,0.0,11
4,1328997,11980517754,JARDINS,Facebook Leads,2024-11-01 00:08:55,Preenchimento,Não está na agenda,NaT,NaN,-,Não é lead,NaN,Não comprou,1,0.0,0,0.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214727,1424698,21976659515,CENTRAL,konquista,2025-01-05 03:00:00+00:00,Botox,Não está na agenda,NaT,NaN,-,Não é lead,NaN,Não comprou,1,0.0,0,0.0,1
214728,1424699,4399652852,CENTRAL,konquista,2025-01-05 03:00:00+00:00,Botox,Não está na agenda,NaT,NaN,-,Não é lead,NaN,Não comprou,1,0.0,0,0.0,1
214729,1424700,11942009519,CENTRAL,konquista,2025-01-05 03:00:00+00:00,Botox,Não está na agenda,NaT,NaN,-,Não é lead,NaN,Não comprou,1,0.0,0,0.0,1
214730,1424701,4497712612,CENTRAL,konquista,2025-01-28 02:56:12.271722+00:00,Indefinido,Não está na agenda,NaT,NaN,-,Não é lead,NaN,Não comprou,1,0.0,0,0.0,1


In [ ]:
df_merged = pd.merge(df_funil_buyers, result2_exploded, left_on='lead_phone', right_on='Telefone', how='left')
df_merged

,lead_id,lead_phone,lead_store,lead_font,lead_created_date,lead_category,appointment_status,appointment_date,appointment_procedure,sales_date,...,Cliente,Email,Telefone,Total Valor Líquido Group by,First Purchase Day Orçamento ID,First Purchase Orçamento ID sum Valor líquido,Last Purchase Day Orçamento ID,Last Purchase Orçamento ID sum Valor líquido,Count Unique Orçamento ID,Average Orçamento Value
0,1329012,11981630412,VILA MASCOTE,Facebook Leads,2024-11-01 00:25:10,Lavieen,Atendido,2024-11-07,AVALIAÇÃO ESTÉTICA,2024-11-07 19:17:39,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
1,1329022,43991146611,LONDRINA,Google Pesquisa,2024-11-01 00:39:37,Cortesia PopUpSaida Peeling_D,Atendido,2024-11-07,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-07 12:48:02,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
2,1329070,11966299163,MOEMA,Facebook Leads,2024-11-01 03:02:14,Lavieen,Atendido,2024-11-07,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-28 19:14:40,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
3,1329097,11986640457,JARDINS,Facebook Leads,2024-11-01 05:46:23,Ultraformer,Atendido,2024-11-01,AVALIAÇÃO ESTÉTICA,2024-11-01 13:08:13,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
4,1329141,11947376615,SÃO BERNARDO,Google Pesquisa,2024-11-01 07:24:14,Lavieen,Atendido,2024-11-07,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-27 16:17:52,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9320,1424347,11944805721,CENTRAL,konquista,2025-01-16 03:00:00+00:00,Preenchimento,Atendido,2025-01-16,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2025-01-16 12:24:38,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
9321,1424429,11911250170,CENTRAL,konquista,2025-01-21 03:00:00+00:00,Indefinido,Não está na agenda,NaT,NaN,2025-01-21 13:51:00,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
9322,1424448,11986240435,CENTRAL,konquista,2025-01-21 03:00:00+00:00,Indefinido,Atendido,2025-01-23,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2025-01-23 18:46:11,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
9323,1424468,11981045454,CENTRAL,konquista,2025-01-21 03:00:00+00:00,Indefinido,Atendido,2025-01-23,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2025-01-23 14:00:51,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN


In [ ]:
# Limpeza dos telefones em result2
result2['Telefone'] = result2['Telefone'].astype(str)  # Garantir que tudo é string
result2['Telefone'] = result2['Telefone'].apply(lambda x: ''.join(filter(str.isdigit, x)))  # Manter apenas dígitos

# Explodir os telefones se necessário
result2['Telefone'] = result2['Telefone'].str.split(',')
result2_exploded = result2.explode('Telefone')

In [ ]:
# Garantir que lead_phone é string e contém apenas dígitos
df_funil_buyers['lead_phone'] = df_funil_buyers['lead_phone'].astype(str)
df_funil_buyers['lead_phone'] = df_funil_buyers['lead_phone'].apply(lambda x: ''.join(filter(str.isdigit, x)))

In [ ]:
# Merge dos dataframes
df_merged = pd.merge(df_funil_buyers, result2_exploded, left_on='lead_phone', right_on='Telefone', how='left')

In [ ]:
df_merged.columns

Index(['lead_id', 'lead_phone', 'lead_store', 'lead_font', 'lead_created_date',
       'lead_category', 'appointment_status', 'appointment_date',
       'appointment_procedure', 'sales_date', 'sales_store', 'sales_value',
       'sales_buyer', 'count', 'sales_value_shared',
       'sales_value_total_by_lead', 'sales_value_total_by_lead_shared', 'mes',
       'Cliente', 'Email', 'Telefone', 'Total Valor Líquido Group by',
       'First Purchase Day Orçamento ID',
       'First Purchase Orçamento ID sum Valor líquido',
       'Last Purchase Day Orçamento ID',
       'Last Purchase Orçamento ID sum Valor líquido',
       'Count Unique Orçamento ID', 'Average Orçamento Value'],
      dtype='object')

In [ ]:
# Filtrar e exibir linhas onde a coluna 'Cliente' é NaN
cliente_nan_rows = df_merged[df_merged['Cliente'].isna()]
cliente_nan_rows.shape

(504, 28)

In [ ]:
# Filtrar e exibir linhas onde a coluna 'Cliente' não é NaN
cliente_not_nan_rows = df_merged[df_merged['Cliente'].notna()]
cliente_not_nan_rows.shape

(9815, 28)

In [ ]:
df_merged.fillna(0, inplace=True)
df_merged

<ipython-input-769-8fd3f5691ac5>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df_merged.fillna(0, inplace=True)


,lead_id,lead_phone,lead_store,lead_font,lead_created_date,lead_category,appointment_status,appointment_date,appointment_procedure,sales_date,...,Cliente,Email,Telefone,Total Valor Líquido Group by,First Purchase Day Orçamento ID,First Purchase Orçamento ID sum Valor líquido,Last Purchase Day Orçamento ID,Last Purchase Orçamento ID sum Valor líquido,Count Unique Orçamento ID,Average Orçamento Value
0,1329012,11981630412,VILA MASCOTE,Facebook Leads,2024-11-01 00:25:10,Lavieen,Atendido,2024-11-07 00:00:00,AVALIAÇÃO ESTÉTICA,2024-11-07 19:17:39,...,Priscila Gislaine Pereira,gislainepri0@gmail.com,11981630412,597.99,2024-11-07 19:17:39,597.99,2024-11-07 19:17:39,597.99,1.0,597.99
1,1329022,43991146611,LONDRINA,Google Pesquisa,2024-11-01 00:39:37,Cortesia PopUpSaida Peeling_D,Atendido,2024-11-07 00:00:00,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-07 12:48:02,...,Daniela Cristina Mello,danimelloo12@hotmail.com,43991146611,50.00,2024-11-07 12:48:02,50.00,2024-11-07 12:48:02,50.00,1.0,50.00
2,1329070,11966299163,MOEMA,Facebook Leads,2024-11-01 03:02:14,Lavieen,Atendido,2024-11-07 00:00:00,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-28 19:14:40,...,Andrea Seixas Pinheiro Sacramento,andreasacra27@gmail.com,11966299163,6900.00,2024-11-07 17:02:53,150.00,2024-12-09 17:11:28,150.00,3.0,2300.00
3,1329097,11986640457,JARDINS,Facebook Leads,2024-11-01 05:46:23,Ultraformer,Atendido,2024-11-01 00:00:00,AVALIAÇÃO ESTÉTICA,2024-11-01 13:08:13,...,Fernanda Papa,fernanda_papa@hotmail.com,11986640457,2529.96,2024-11-01 13:08:13,2529.96,2024-11-01 13:08:13,2529.96,1.0,2529.96
4,1329141,11947376615,SÃO BERNARDO,Google Pesquisa,2024-11-01 07:24:14,Lavieen,Atendido,2024-11-07 00:00:00,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-27 16:17:52,...,Maria emiko iuvata,emiko.ayna@gmail.com,11947376615,693.00,2024-11-27 16:17:52,597.00,2024-11-29 16:46:18,96.00,2.0,346.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10314,1424347,11944805721,CENTRAL,konquista,2025-01-16 03:00:00+00:00,Preenchimento,Atendido,2025-01-16 00:00:00,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2025-01-16 12:24:38,...,Diana Veronica,moranjuanmayor@gmail.com,11944805721,850.00,2025-01-16 12:24:38,850.00,2025-01-16 12:24:38,850.00,1.0,850.00
10315,1424429,11911250170,CENTRAL,konquista,2025-01-21 03:00:00+00:00,Indefinido,Não está na agenda,0,0,2025-01-21 13:51:00,...,Camilla Rocha Franca,camilla.franca@poli.ufrj.br,11911250170,90.00,2025-01-21 13:51:00,90.00,2025-01-21 13:51:00,90.00,1.0,90.00
10316,1424448,11986240435,CENTRAL,konquista,2025-01-21 03:00:00+00:00,Indefinido,Atendido,2025-01-23 00:00:00,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2025-01-23 18:46:11,...,Giullia Meira Zanchi,Giullia.zancchi@gmail.com,11986240435,900.00,2025-01-23 18:46:11,900.00,2025-01-23 18:46:11,900.00,1.0,900.00
10317,1424468,11981045454,CENTRAL,konquista,2025-01-21 03:00:00+00:00,Indefinido,Atendido,2025-01-23 00:00:00,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2025-01-23 14:00:51,...,Rafaela Aparecida Derrico Moraes da Silva,rafaderrico@hotmail.com,11981045454,780.00,2025-01-23 14:00:51,780.00,2025-01-23 14:00:51,780.00,1.0,780.00


In [ ]:
df_funil_completo = pd.concat([df_merged, df_funil_non_buyers], ignore_index=True)
df_funil_completo

,lead_id,lead_phone,lead_store,lead_font,lead_created_date,lead_category,appointment_status,appointment_date,appointment_procedure,sales_date,...,Cliente,Email,Telefone,Total Valor Líquido Group by,First Purchase Day Orçamento ID,First Purchase Orçamento ID sum Valor líquido,Last Purchase Day Orçamento ID,Last Purchase Orçamento ID sum Valor líquido,Count Unique Orçamento ID,Average Orçamento Value
0,1329012,11981630412,VILA MASCOTE,Facebook Leads,2024-11-01 00:25:10,Lavieen,Atendido,2024-11-07 00:00:00,AVALIAÇÃO ESTÉTICA,2024-11-07 19:17:39,...,Priscila Gislaine Pereira,gislainepri0@gmail.com,11981630412,597.99,2024-11-07 19:17:39,597.99,2024-11-07 19:17:39,597.99,1.0,597.99
1,1329022,43991146611,LONDRINA,Google Pesquisa,2024-11-01 00:39:37,Cortesia PopUpSaida Peeling_D,Atendido,2024-11-07 00:00:00,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-07 12:48:02,...,Daniela Cristina Mello,danimelloo12@hotmail.com,43991146611,50.00,2024-11-07 12:48:02,50.00,2024-11-07 12:48:02,50.00,1.0,50.00
2,1329070,11966299163,MOEMA,Facebook Leads,2024-11-01 03:02:14,Lavieen,Atendido,2024-11-07 00:00:00,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-28 19:14:40,...,Andrea Seixas Pinheiro Sacramento,andreasacra27@gmail.com,11966299163,6900.00,2024-11-07 17:02:53,150.00,2024-12-09 17:11:28,150.00,3.0,2300.00
3,1329097,11986640457,JARDINS,Facebook Leads,2024-11-01 05:46:23,Ultraformer,Atendido,2024-11-01 00:00:00,AVALIAÇÃO ESTÉTICA,2024-11-01 13:08:13,...,Fernanda Papa,fernanda_papa@hotmail.com,11986640457,2529.96,2024-11-01 13:08:13,2529.96,2024-11-01 13:08:13,2529.96,1.0,2529.96
4,1329141,11947376615,SÃO BERNARDO,Google Pesquisa,2024-11-01 07:24:14,Lavieen,Atendido,2024-11-07 00:00:00,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-27 16:17:52,...,Maria emiko iuvata,emiko.ayna@gmail.com,11947376615,693.00,2024-11-27 16:17:52,597.00,2024-11-29 16:46:18,96.00,2.0,346.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215721,1424698,21976659515,CENTRAL,konquista,2025-01-05 03:00:00+00:00,Botox,Não está na agenda,NaT,NaN,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215722,1424699,4399652852,CENTRAL,konquista,2025-01-05 03:00:00+00:00,Botox,Não está na agenda,NaT,NaN,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215723,1424700,11942009519,CENTRAL,konquista,2025-01-05 03:00:00+00:00,Botox,Não está na agenda,NaT,NaN,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215724,1424701,4497712612,CENTRAL,konquista,2025-01-28 02:56:12.271722+00:00,Indefinido,Não está na agenda,NaT,NaN,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_funil_completo.fillna(0, inplace=True)
df_funil_completo

,lead_id,lead_phone,lead_store,lead_font,lead_created_date,lead_category,appointment_status,appointment_date,appointment_procedure,sales_date,...,Cliente,Email,Telefone,Total Valor Líquido Group by,First Purchase Day Orçamento ID,First Purchase Orçamento ID sum Valor líquido,Last Purchase Day Orçamento ID,Last Purchase Orçamento ID sum Valor líquido,Count Unique Orçamento ID,Average Orçamento Value
0,1329012,11981630412,VILA MASCOTE,Facebook Leads,2024-11-01 00:25:10,Lavieen,Atendido,2024-11-07 00:00:00,AVALIAÇÃO ESTÉTICA,2024-11-07 19:17:39,...,Priscila Gislaine Pereira,gislainepri0@gmail.com,11981630412,597.99,2024-11-07 19:17:39,597.99,2024-11-07 19:17:39,597.99,1.0,597.99
1,1329022,43991146611,LONDRINA,Google Pesquisa,2024-11-01 00:39:37,Cortesia PopUpSaida Peeling_D,Atendido,2024-11-07 00:00:00,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-07 12:48:02,...,Daniela Cristina Mello,danimelloo12@hotmail.com,43991146611,50.00,2024-11-07 12:48:02,50.00,2024-11-07 12:48:02,50.00,1.0,50.00
2,1329070,11966299163,MOEMA,Facebook Leads,2024-11-01 03:02:14,Lavieen,Atendido,2024-11-07 00:00:00,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-28 19:14:40,...,Andrea Seixas Pinheiro Sacramento,andreasacra27@gmail.com,11966299163,6900.00,2024-11-07 17:02:53,150.00,2024-12-09 17:11:28,150.00,3.0,2300.00
3,1329097,11986640457,JARDINS,Facebook Leads,2024-11-01 05:46:23,Ultraformer,Atendido,2024-11-01 00:00:00,AVALIAÇÃO ESTÉTICA,2024-11-01 13:08:13,...,Fernanda Papa,fernanda_papa@hotmail.com,11986640457,2529.96,2024-11-01 13:08:13,2529.96,2024-11-01 13:08:13,2529.96,1.0,2529.96
4,1329141,11947376615,SÃO BERNARDO,Google Pesquisa,2024-11-01 07:24:14,Lavieen,Atendido,2024-11-07 00:00:00,AVALIAÇÃO INJETÁVEIS E INVASIVOS,2024-11-27 16:17:52,...,Maria emiko iuvata,emiko.ayna@gmail.com,11947376615,693.00,2024-11-27 16:17:52,597.00,2024-11-29 16:46:18,96.00,2.0,346.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215721,1424698,21976659515,CENTRAL,konquista,2025-01-05 03:00:00+00:00,Botox,Não está na agenda,0,0,-,...,0,0,0,0.00,0,0.00,0,0.00,0.0,0.00
215722,1424699,4399652852,CENTRAL,konquista,2025-01-05 03:00:00+00:00,Botox,Não está na agenda,0,0,-,...,0,0,0,0.00,0,0.00,0,0.00,0.0,0.00
215723,1424700,11942009519,CENTRAL,konquista,2025-01-05 03:00:00+00:00,Botox,Não está na agenda,0,0,-,...,0,0,0,0.00,0,0.00,0,0.00,0.0,0.00
215724,1424701,4497712612,CENTRAL,konquista,2025-01-28 02:56:12.271722+00:00,Indefinido,Não está na agenda,0,0,-,...,0,0,0,0.00,0,0.00,0,0.00,0.0,0.00


In [ ]:
funil = 'https://docs.google.com/spreadsheets/d/1g2nLbAg-JxilBST3xWbOOyu-M7IRTBO-MYyWN1vDl7A/edit?gid=0#gid=0'

spreadsheet = gc.open_by_url(funil) # MUDAR O LINK DA PLANILHA AQUI

# Selecting desired sheet
sheet_data = spreadsheet.worksheet('data')

sheet_data.clear()

# Colar o DataFrame na aba especificada
set_with_dataframe(sheet_data, df_funil_completo)